<a href="https://colab.research.google.com/github/Fernando-codes/ia-trading-predictivo/blob/main/Lista_despl_B_1%2C2%2C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
def detectar_temporalidades(data):
    """
    Retorna una lista con las temporalidades detectadas ('15M', '1H', '4H') según claves en 'data'.
    """
    return [tf for tf in ["15M", "1H", "4H"] if any(tf in k for k in data.keys())]


In [34]:
# ===========================
# 1. SCRIPT BLOQUE 1 – VERSIÓN FINAL
# ===========================


def analizar_hoja(data_raw):

    import re
    import pandas as pd
    from IPython.display import display, HTML
    import textwrap
    import inspect

    def parse_float(val):
        if val in (None, "∅", ""):
            return None
        try:
            return float(str(val).replace(",", "."))
        except (ValueError, TypeError):
            return None

    # --- NUEVO BLOQUE: Transforma texto plano en dict ---
    if isinstance(data_raw, str):
        # Convierte texto plano de pares clave/valor en un dict
        lineas = [x.strip() for x in data_raw.strip().splitlines() if x.strip()]
        data = {}
        for i in range(0, len(lineas)-1, 2):
            clave = lineas[i]
            valor = lineas[i+1]
            data[clave] = valor
    else:
        data = data_raw

    # — Asegúrate de definir e inyectar estos niveles BOS y CHoCH para todas las TF —
    for tf in ["15M", "1H", "4H"]:
        # Claves proyectadas
        k_bos_a = f"Nivel_Proy_BOS_Alcista_{tf}"
        k_bos_b = f"Nivel_Proy_BOS_Bajista_{tf}"
        k_cho_a = f"Nivel_Proy_CHOCH_Alcista_{tf}"
        k_cho_b = f"Nivel_Proy_CHOCH_Bajista_{tf}"

        # Parseo a float (None si no existe o no es numérico)
        v_bos_a = parse_float(data.get(k_bos_a))
        v_bos_b = parse_float(data.get(k_bos_b))
        v_cho_a = parse_float(data.get(k_cho_a))
        v_cho_b = parse_float(data.get(k_cho_b))

        # Reinyecta en data para que raw_BOS/CHOCH_PROY_DIST los encuentre
        data[k_bos_a] = v_bos_a
        data[k_bos_b] = v_bos_b
        data[k_cho_a] = v_cho_a
        data[k_cho_b] = v_cho_b





    # 1.1.1 — Soporte para DataFrame: si te pasan un pd.DataFrame,
    # lo convertimos al texto clave/valor que la función espera.
    if isinstance(data_raw, pd.DataFrame):
        filas = []
        # Tomamos las dos primeras columnas como clave y valor
        for k, v in data_raw.iloc[:, :2].itertuples(index=False):
            filas.append(str(k).strip())
            filas.append(str(v).strip())
        data_raw = "\n".join(filas)




    # 1.3 Parseo de data_raw → diccionario 'data'
    # 1.3.1 Separación en pares clave/valor y setup inicial
    lines = [l.strip() for l in data_raw.splitlines() if l.strip()]
    pairs = list(zip(lines[0::2], lines[1::2]))

    data = {}
    current_tf = None

    for k, raw_val in pairs:
        raw_val = raw_val.replace("−", "-").replace("–", "-")
        k = k.strip()
        k = k.replace(" ", "_").replace(".", "")               # espacios → _ y puntos fuera
        k = k.replace("-", "_").replace("–", "_")              # guiones → _
        k = re.sub(r"__+", "_", k)                             # doble guión bajo → simple
        k = re.sub(r"_\([^)]*\)$", "", k)                      # elimina sufijos como _(dentro)



    # 1.3.2 Detección de temporalidades
        if k.lower() == "tf":
            current_tf = raw_val.strip().upper()
            continue

    # 1.3.3 Extracción de Trend Bias con categoría
        if k.lower().startswith("trend_bias_") and any(cat in k for cat in [
            "Tendencia_Alcista", "Tendencia_Bajista",
            "Lateral_sesgo_Alcista", "Lateralidad_sin_sesgo", "Lateral_sesgo_Bajista"
        ]):
            s = str(raw_val)
            num = re.sub(r"[^\d\-.,]", "", s)
            if re.search(r"\d", num):
                if num.count(",") == 1 and num.count(".") >= 1:
                    num = num.replace(".", "").replace(",", ".")
                elif num.count(",") == 1 and num.count(".") == 0:
                    num = num.replace(",", ".")
                elif num.count(".") > 1 and num.count(",") == 0:
                    num = num.replace(".", "")
                try:
                    data[k] = float(num)
                    continue
                except ValueError:
                    pass
            data[k] = s.strip()
            continue

    # 1.3.3 Extracción de Trend Bias con categoría
        if k.lower() in ("dirección", "direccion"):
            if current_tf:
                data[f"direccion_{current_tf}"] = raw_val.strip()
            else:
                data["direccion"] = raw_val.strip()
            continue

    # 1.3.5 Identificación de activo
        if k.lower() == "activo":
            data[k] = raw_val.strip()
            continue

    # 1.3.6 Conversión genérica de valores numéricos o string
        s = str(raw_val)
        num = re.sub(r"[^\d\-.,]", "", s)
        if re.search(r"\d", num):
            if num.count(",") == 1 and num.count(".") >= 1:
                num = num.replace(".", "").replace(",", ".")
            elif num.count(",") == 1 and num.count(".") == 0:
                num = num.replace(",", ".")
            elif num.count(".") > 1 and num.count(",") == 0:
                num = num.replace(".", "")
            try:
                data[k] = float(num)
                continue
            except ValueError:
                pass

        data[k] = s.strip()
    # 1.3.7 Función detectar_temporalidades(data)
    def detectar_temporalidades(data):
        """
        Retorna una lista con las temporalidades detectadas ('15M', '1H', '4H') según claves en 'data'.
        """
        return [tf for tf in ["15M", "1H", "4H"] if any(tf in k for k in data.keys())]




    # ————————————————————————————————————————
    # 1.4 CÁLCULO DE INDICADORES DERIVADOS NECESARIOS
    # ————————————————————————————————————————
    # Asegúrate de que la función detectar_temporalidades ya está definida.
    for tf in detectar_temporalidades(data):
        # 1.4.1 Cálculo de BB_Width_percent
        hi  = data.get(f"BB_Upper_{tf}")
        lo  = data.get(f"BB_Lower_{tf}")
        mid = data.get(f"Mid_Range_{tf}")
        if all(isinstance(x, (int, float)) for x in (hi, lo, mid)) and mid != 0:
            data[f"BB_Width_percent_{tf}"] = (hi - lo) / mid
        else:
            data[f"BB_Width_percent_{tf}"] = 0.0

        # --- CORRECCIÓN FORMATO BBWIDTH ---
        bw_raw = data.get(f"BB_Width_%_{tf}")
        if bw_raw is not None:
            if abs(bw_raw) > 0.1:
                data[f"BBWIDTH_{tf}"] = bw_raw / 100
            else:
                data[f"BBWIDTH_{tf}"] = bw_raw
        else:
            data[f"BBWIDTH_{tf}"] = data.get(f"BB_Width_percent_{tf}", 0.0)



        # 1.4.2 Cálculo de Dist_HTFHigh_10pct
        rt       = data.get("Valor_mcdo")
        marco_hi = data.get(f"Marco_High_{tf}")
        marco_lo = data.get(f"Marco_Low_{tf}")
        if all(isinstance(x, (int, float)) for x in (rt, marco_hi, marco_lo)):
            rango = marco_hi - marco_lo
            if rango > 0:
        # 1.4.3 distancia desde rt hasta el high como ratio sobre 10% de rango
                data[f"Dist_HTFHigh_10pct_{tf}"] = abs(marco_hi - rt) / (rango * 0.10)
            else:
                data[f"Dist_HTFHigh_10pct_{tf}"] = 0.0
        else:
            data[f"Dist_HTFHigh_10pct_{tf}"] = 0.0

    # 1.4.3 Cálculo de ZONACENTRAL y REBOTEEXTREMO para cada TF detectada
    for tf in detectar_temporalidades(data):
        rt  = data.get("Valor_mcdo")
        hi  = data.get(f"Marco_High_{tf}")
        lo  = data.get(f"Marco_Low_{tf}")
        mid = data.get(f"Mid_Range_{tf}")

        # --- ZONACENTRAL ---
        rango = hi - lo if hi is not None and lo is not None else None
        if all(isinstance(x, (int, float)) for x in (rt, mid, rango)) and rango > 0:
            data[f"ZONACENTRAL_{tf}"] = 100 * (1 - abs(rt - mid) / (rango / 2))
        else:
            data[f"ZONACENTRAL_{tf}"] = 0.0

        # --- REBOTEEXTREMO ---
        bb  = data.get(f"BB Width %_{tf}", 0.0)
        rsi = data.get(f"RSI_{tf}", 0.0)
        vol = data.get(f"VolRel_{tf}", 0.0)
        data[f"REBOTEEXTREMO_{tf}"] = 1.0 if (bb < 0.006 and rsi < 30 and vol >= 1.0) else 0.0




    # =========================================================
    # 1.5 – CHEQUEO DE INTEGRIDAD DE CAMPOS
    # =========================================================
    import pandas as pd
    from IPython.display import display, HTML
    import re, textwrap

    # 1.5.1 ─ Indicadores esperados
    INDICADORES_ESPERADOS = {
        "Valor_mcdo"            : False,
        "Marco_High"            : True,
        "Marco_Low"             : True,
        "Mid_Range"             : True,
        "ATR"                   : True,
        "RSI"                   : True,
        "ADX"                   : True,
        "OBV_Score"             : True,
        "EMA_Cross"             : True,
        "MACD_Line"             : True,
        "MACD_Signal"           : True,
        "MACD_Hist"             : True,
        "BB_Width_percent"      : True,
        "VolRel"                : True,
        "Dist_ATH_LE_ATR"       : True,
        "Dist_HTFHigh_10pct"    : True,
        "Fibo_zona_envolvente"  : True,
        "Nivel_Proy_BOS_Alcista": True,
        "Nivel_Proy_BOS_Bajista": True,
    }


    # 1.5.2 ─ Temporalidades presentes
    TFS = [t for t in ("15M", "1H", "4H") if any(k.endswith(f"_{t}") for k in data)]

    # — INYECCIÓN / NORMALIZACIÓN DE FIBO_ENV (zona envolvente 78.6–100%) —
    for tf in TFS:
        raw = data.get(f"Fibo_Retr_{tf}", 0.0)
        try:
            val = float(str(raw).replace(",", "."))
        except (TypeError, ValueError):
            val = 0.0

        # Rango válido para zona envolvente
        dentro_env = 0.786 <= val <= 1.0

        # Guarda el valor crudo y si está dentro del rango
        data[f"FIBO_ENV_{tf}"]                      = val
        data[f"Fibo_zona_envolvente_{tf}"]          = val
        data[f"Fibo_zona_envolvente_{tf} (dentro)"] = dentro_env

    # — INYECCIÓN / NORMALIZACIÓN DE FIBO 38–61 % (retroceso) —
    for tf in TFS:
        raw = data.get(f"Fibo_retroceso_{tf}")
        try:
            val = float(str(raw).replace(",", "."))
        except (TypeError, ValueError):
            val = 0.0

        # Normalización y reinyección
        data[f"FIBO_{tf}"] = 1.0 if 0.382 <= val <= 0.618 else 0.0
        data[f"Fibo_retroceso_{tf}"] = val



    # 1.5.3 ─ Comparación vs. lo presente
    claves_presentes = set(data.keys())
    missing_rows, dup_rows = [], []

    for base, usa_tf in INDICADORES_ESPERADOS.items():
        if usa_tf:
            for tf in TFS:
                patron = re.compile(fr"^{base}(_\d+[MH])(_.*)?$", re.I)
                hits = [k for k in claves_presentes if patron.match(k) and k.endswith(f"_{tf}")]
                if not hits:
                    missing_rows.append({"Indicador": f"{base}_{tf}", "Problema": "❌ Falta"})
                elif len(hits) > 1:
                    dup_rows.append({
                        "Indicador": f"{base}_{tf}",
                        "Problema" : "⚠️ Duplicado / variantes",
                        "Coincidencias": ", ".join(hits)
                    })
        else:
            if base not in claves_presentes:
                missing_rows.append({"Indicador": base, "Problema": "❌ Falta"})

    # 1.5.4 ─ Informe visual
    def show_df(rows, title, color):
        if rows:
            df = pd.DataFrame(rows)
            display(HTML(f"<h3 style='color:{color}'>{title}</h3>"))
            display(df.style.hide(axis='index'))
        else:
            display(HTML(f"<p style='color:green'><b>{title}: ✔ Ninguno</b></p>"))

    show_df(missing_rows, "Indicadores ausentes", "#C0392B")
    show_df(dup_rows,     "Indicadores duplicados / con sufijos inesperados", "#D68910")

    # 1.5.5 ─ Freno si hay faltantes críticos
    if any(r["Problema"].startswith("❌") for r in missing_rows):
        raise RuntimeError("👉 Revisa los indicadores ausentes antes de continuar.")







    # 1.6 PARÁMETROS BASE =======================================================
    # 1.6.1 # 1.6.1 Parámetros umbral ADX
    umbral_adx_estructura  = 20
    umbral_adx_momentum    = 25

    # 1.7 UTILIDADES GENERALES ==================================================

    # 1.7.1 Formato de número con coma europea
    def formato_numero(x):
        try:
            return f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
        except:
            return x
    # 1.7.2 Distancia en $ y validación por ATR
    def formato_dist_dolar(rt, hi, lo, mid, atr, bias):
        if not all(isinstance(u, (int, float)) for u in (rt, hi, lo, mid, atr)):
            return "-"
        bias = bias.lower()
        if "tendencia alcista" in bias:
            dist = (hi - rt) / atr
            return f"$ → High: {round(dist,1)} ATR {'✔' if dist >= 1 else '✘'}"
        if "tendencia bajista" in bias:
            dist = (rt - lo) / atr
            return f"$ → Low: {round(dist,1)} ATR {'✔' if dist >= 1 else '✘'}"
        if "lateralidad con sesgo alcista" in bias:
            dist = (rt - (mid - 0.5*atr)) / atr
            signo = "+" if dist >= 0 else "−"
            return f"MRange → $: {signo}{round(dist,1)} ATR {'✔' if dist >= 0.5 else '✘'}"
        if "lateralidad con sesgo bajista" in bias:
            dist = ((mid + 0.5*atr) - rt) / atr
            signo = "−" if dist >= 0 else "+"
            return f"MRange → $: {signo}{round(dist,1)} ATR {'✔' if dist >= 0.5 else '✘'}"
        return "-"
    # 1.7.3 Visualización ADX con ✓/✗
    def f_symbolADX(v):
        """
        Devuelve el ADX numérico y un ✓ / ✗ **sin colorear**.
        Uso ✓ (U+2713) y ✗ (U+2717) para que la función `pintar`
        no los envuelva en <span style='color:…'>.
        """
        if not isinstance(v, (int, float)):
            return f"{v}\n✗"
        signo = "✓" if v >= umbral_adx_estructura else "✗"
        return f"{formato_numero(v)}\n{signo}"
    # 1.7.4 Visualización OBV con ✓/✗
    def f_symbolOBV(v):
        return (f"{int(v)}\n{'✔' if v == 1 else '✘'}"
                if isinstance(v, (int, float)) else f"{v}\n✘")
    # 1.7.5 Visualización EMA estructural con ✓/✗
    def f_symbolEMA_estructura(v):
        try:
            return "+1\n✔" if v > 0 else ("-1\n✘" if v < 0 else "0\n✘")
        except:
            return f"{v}\n✘"
    # 1.7.6 Visualización EMA proyectado con ✓/✗
    def f_symbolEMA_proyectado(v):
        try:
            return "+1\n✔" if v > 0 else ("-1\n✘" if v < 0 else "0\n✘")
        except:
            return f"{v}\n✘"
    # 1.7.7 Visualización RSI ajustado por contexto
    def f_symbolRSI(v, bias=""):
        if not isinstance(v, (int, float)):
            return f"{v}\n✘"
        bias = bias.lower()
        if "lateralidad con sesgo alcista" in bias:
            return f"{round(v,1)}\n{'✔' if v > 60 else '✘'}"
        elif "lateralidad con sesgo bajista" in bias:
            return f"{round(v,1)}\n{'✔' if v < 40 else '✘'}"
        else:
            return f"{round(v,1)}\n{'✔' if 30 <= v <= 70 else '✘'}"


    # —————————————————————————————————————————————————
    # Función auxiliar para EXTRAER el tipo de entrada
    # — recibe el texto de estado y devuelve "táctica", "agresiva" o "conservadora"
    def extraer_tipo_entrada(estado: str) -> str:
        est = estado.lower()
        if "táctica validada" in est or "entrada táctica validada" in est or "táctica" in est:
            return "táctica"
        elif "agresiva" in est:
            return "agresiva"
        else:
            # si ni táctica ni agresiva, caerá aquí; puedes ajustar la lógica según tus casos
            return "conservadora"
    # —————————————————————————————————————————————————

    # 1.8 UMBRAL DINÁMICO POR VOLATILIDAD =======================================
    def umbral_por_vol(atr, lo, hi):
        if atr >= 7:
            return 1.0
        rango = hi - lo + 1e-6
        atr_ratio = atr / rango
        if atr_ratio < 0.4:
            return 2.0
        elif atr_ratio < 0.7:
            return 1.5
        else:
            return 1.0

    # 1.9 FUNCIONES AUXILIARES LÓGICA ===========================================
    # 1.9.1 Validación táctica en tendencia alcista
    def validar_tactico_alcista(d, tf="1H"):
        rt  = d["Valor_mcdo"]
        hi  = d[f"Marco_High_{tf}"]
        lo  = d[f"Marco_Low_{tf}"]
        atr = d[f"ATR_{tf}"]
        obv = d[f"OBV_Score_{tf}"]
        ema = d[f"EMA_Cross_{tf}"]
        adx = d[f"ADX_{tf}"]

        pts = 0.0
        razones = []
        if obv == 1:
            pts += 1.0
        else:
            razones.append("OBV ≠ 1")
        if (hi - rt) >= atr:
            pts += 1.0
        else:
            razones.append("Distancia a High < 1×ATR")
        if ema == 1:
            pts += 0.5
        else:
            razones.append("EMA_Cross ≠ +1")
        umbral = umbral_por_vol(atr, lo, hi)
        if pts >= umbral:
            return {"validado": True, "estado": "Tendencia alcista validada: entrada táctica con frame estructural"}
        if (hi - rt) >= atr and adx >= 30:
            return {"validado": False, "estado": "Entrada agresiva en tendencia alcista con frame estructural: Fuerza tendencia y buen rango de operación"}
        return {"validado": False, "estado": f"🔴 No validado por puntaje ({'; '.join(razones)})."}

    # 1.9.2 Validación táctica en tendencia bajista
    def validar_tactico_bajista(d, tf="1H"):
        rt  = d["Valor_mcdo"]
        hi  = d[f"Marco_High_{tf}"]
        lo  = d[f"Marco_Low_{tf}"]
        atr = d[f"ATR_{tf}"]
        obv = d[f"OBV_Score_{tf}"]
        ema = d[f"EMA_Cross_{tf}"]
        adx = d[f"ADX_{tf}"]

        pts = 0.0
        razones = []
        if obv == -1:
            pts += 1.0
        else:
            razones.append("OBV ≠ -1")
        if (rt - lo) >= atr:
            pts += 1.0
        else:
            razones.append("Distancia a Low < 1×ATR")
        if ema == -1:
            pts += 0.5
        else:
            razones.append("EMA_Cross ≠ -1")
        umbral = umbral_por_vol(atr, lo, hi)
        if pts >= umbral:
            return {"validado": True, "estado": "Tendencia bajista validada: entrada táctica con frame estructural"}
        if (rt - lo) >= atr and adx >= 30:
            return {"validado": False, "estado": "Entrada agresiva en tendencia bajista con frame estructural: Fuerza tendencia y buen rango de operación"}
        return {"validado": False, "estado": f"🔴 No validado por puntaje ({'; '.join(razones)})."}

    # === 1.10 Validaciones de lateralidad y proyección ==========================
    # 1.10.1 Validación táctica: lateralidad con sesgo alcista
    def validar_lateralidad_alcista(d, tf="1H"):
        rt  = d["Valor_mcdo"]
        mid = d[f"Mid_Range_{tf}"]
        atr = d[f"ATR_{tf}"]
        obv = d[f"OBV_Score_{tf}"]
        rsi = d[f"RSI_{tf}"]

        ok = 0
        if rsi > 60:  ok += 1                 # señal ① RSI
        if obv == 1:  ok += 1                 # señal ② OBV
        if (rt - (mid - 0.5*atr)) / atr >= 0.5:  ok += 1  # señal ③ Distancia

        if ok >= 2:
            estado = (f"Entrada táctica validada con {ok}/3 señales para "
                      "Lateralidad sesgo alcista con Frame estructural – "
                      "presión sobre resistencia estructural")
            return {"validado": True, "ok": ok, "estado": estado}
        estado = f"Entrada táctica no validada, solo {ok}/3 señal(es) favorables"
        return {"validado": False, "ok": ok, "estado": estado}

    # 1.10.2 Validación táctica: lateralidad con sesgo bajista
    def validar_lateralidad_bajista(d, tf="1H"):
        rt  = d["Valor_mcdo"]
        mid = d[f"Mid_Range_{tf}"]
        atr = d[f"ATR_{tf}"]
        obv = d[f"OBV_Score_{tf}"]
        rsi = d[f"RSI_{tf}"]

        ok = 0
        if rsi < 40:  ok += 1                 # señal ① RSI
        if obv == -1: ok += 1                 # señal ② OBV
        if ((mid + 0.5*atr) - rt) / atr >= 0.5:  ok += 1  # señal ③ Distancia

        if ok >= 2:
            estado = (f"Entrada táctica validada con {ok}/3 señales para "
                      "Lateralidad sesgo bajista con Frame estructural – "
                      "presión sobre soporte estructural")
            return {"validado": True, "ok": ok, "estado": estado}
        estado = f"Entrada táctica no validada, solo {ok}/3 señal(es) favorables"
        return {"validado": False, "ok": ok, "estado": estado}

    # 1.10.3 Evaluación de frame estructural
    def evaluar_frame_estructural(d, tf="1H"):
        rt  = d.get("Valor_mcdo")
        hi  = d.get(f"Marco_High_{tf}")
        lo  = d.get(f"Marco_Low_{tf}")
        atr = d.get(f"ATR_{tf}")
        if not all(isinstance(x, (int, float)) for x in (rt, hi, lo, atr)):
            return False
        return (lo + 0.75*atr) <= rt <= (hi - 0.75*atr)

    # 1.10.4 Detección y diagnóstico de frame proyectado
    def determinar_frame_proyectado(d, tf="1H"):
        rt    = d["Valor_mcdo"]
        hi    = d[f"Marco_High_{tf}"]
        lo    = d[f"Marco_Low_{tf}"]
        mid   = d[f"Mid_Range_{tf}"]
        ath   = d.get(f"All_Time_High_{tf}", float("inf"))
        atr   = d[f"ATR_{tf}"]
        adx   = d[f"ADX_{tf}"]
        obv   = d[f"OBV_Score_{tf}"]
        rango = hi - lo + 1e-6

        # — EMA real según temporalidad —
        if   tf == "15M":
            ema_price = d.get("EMA20_15M")
        elif tf == "1H":
            ema_price = d.get("EMA50_1H")
        else:  # "4H"
            ema_price = d.get("EMA200_4H")
        # fallback si falta
        if ema_price is None:
            ema_price = hi

        # ─── CASO A: conservadora vs envolvente ────────────────────────────────────
        if lo <= rt <= hi and not ((lo + 0.75*atr) <= rt <= (hi - 0.75*atr)):
            # — Zona operativa Envolvente —
            if (hi - rt) <= atr:
                tipo_zona = "Zona operativa Envolvente"
                f_low     = mid
                factor    = 1.5 if tf in ("15M","1H") else 2.0
                f_high    = max(hi + 1.0, ema_price + factor*atr)
                diag      = "Diagnóstico: presión sin impulso suficiente → entrada táctica limitada. Proyección zona envolvente"
            # — Zona proyección conservadora —
            elif (rt - lo) <= atr:
                tipo_zona = "Zona operativa proyección conservadora"
                f_low     = lo - 1.5*atr
                f_high    = mid
                diag      = "Presión activa: posibilidad de entrada agresiva. Zona operativa proyección conservadora"
            # — Zona operativa Clásica —
            else:
                tipo_zona = "Zona operativa Clásica"
                f_low     = lo
                f_high    = hi
                diag      = "Sin espacio operable claro → posible zona agotada"

        else:
            # ─── CASO B: anticipada o crítica ───────────────────────────────────────
            if lo <= rt <= hi and (ath - rt) <= atr:
                if adx > umbral_adx_momentum:
                    tipo_zona = "Proyección anticipada"
                    f_low     = mid
                    f_high    = hi + (hi - lo)*1.618
                    diag      = "Presión activa: posibilidad de entrada agresiva. Confirmación para operar con Proyección anticipada"
                else:
                    tipo_zona = "Zona crítica con momentum débil"
                    f_low     = mid
                    factor    = 1.5 if tf in ("15M","1H") else 2.0
                    f_high    = max(hi + 1.0, ema_price + factor*atr)
                    diag      = "Zona crítica con momentum débil. Diagnóstico: presión sin impulso suficiente → entrada táctica limitada. Proyección zona envolvente"
            # ─── CASO C: rompimiento de ATH ────────────────────────────────────────
            elif rt > ath:
                if adx > umbral_adx_momentum:
                    tipo_zona = "Proyección zona operativa completa"
                    f_low     = mid
                    f_high    = hi + (hi - lo)*1.618
                    diag      = "Tipo de entrada: Proyección zona operativa completa. Activación tras rompimiento de ATH con momentum técnico → expansión con Fibo 1.618"
                else:
                    tipo_zona = "Proyección zona envolvente (rompimiento débil)"
                    f_low     = mid
                    factor    = 1.5 if tf in ("15M","1H") else 2.0
                    f_high    = max(hi + 1.0, ema_price + factor*atr)
                    diag      = "Tipo de entrada: Proyección zona envolvente (rompimiento débil). Diagnóstico: expansión táctica limitada por falta de momentum técnico"
            # — Sin oportunidad proyectiva —
            else:
                tipo_zona = "Sin oportunidad proyectiva"
                f_low     = lo
                f_high    = hi
                diag      = "No validado: sin oportunidad proyectiva"

        # ─── CASO E: proximidad ATL ───────────────────────────────────────────────
        if abs(rt - lo) <= atr:
            tipo_zona = "Zona operativa proyectada conservadora (proximidad ATL)"
            diag      = "Presión activa: posibilidad de entrada agresiva. Zona operativa proyectada conservadora (proximidad ATL)"

        # EXTRA: guardar dirección proyectada para uso en Bloque 2
        if "tendencia" in diag.lower():
            if "alcista" in diag.lower():
                direccion_detectada = "Tendencia alcista"
            elif "bajista" in diag.lower():
                direccion_detectada = "Tendencia bajista"
            else:
                direccion_detectada = "Tendencia indefinida"
        elif "lateralidad" in diag.lower() or "lateral" in diag.lower():
            if "alcista" in diag.lower():
                direccion_detectada = "Lateralidad con sesgo alcista"
            elif "bajista" in diag.lower():
                direccion_detectada = "Lateralidad con sesgo bajista"
            else:
                direccion_detectada = "Lateralidad sin sesgo"
        else:
            direccion_detectada = "Dirección no determinada"

        return {
            "tipo_zona": tipo_zona,
            "f_low":     formato_numero(f_low),
            "f_high":    formato_numero(f_high),
            "rango_pts": formato_numero(f_high - f_low),
            "atr":       formato_numero(atr),
            "prox":      "",      # si lo usas deja aquí el valor
            "diag":      diag,
            "direccion_detectada": direccion_detectada  # ← CLAVE para Bloque 2
        }




    # 1.11 UTILIDADES Y FORMATOS DE SALIDA =====================================
    # 1.11.1 DETECTOR DE TEMPORALIDADES ===========================================
    def detectar_temporalidades(d):
        return [t for t in ("15M","1H","4H") if any(k.endswith(f"_{t}") for k in d)]

    # 1.11.2 NUEVA UTILIDAD: Formato R & Score ================================
    def formato_R_score(raw, score):
        """
        Recibe un valor raw (crudo) y su score numérico,
        y lo formatea en dos líneas:
          R: <raw>
          Score: <score>
        """
        return f"R: {raw}\\nScore: {score}"

    # 1.11.3 Formato R & Score (versión simple) =====================================
    def formato_R_score(raw, score):
        return f"R: {raw}\nScore: {score}"

    # 1.11.4 Visualización de tabla en HTML ================================================
    def mostrar_tabla(df, header_bg="#f0f8ff", header_fg="#1f497d", cell_border_color="#444"):
        def pintar(v):
            return str(v).replace("✔", "<span style='color:green;font-weight:bold'>✔</span>").replace("✘", "<span style='color:red;font-weight:bold'>✘</span>").replace("\n", "<br>")
        css_hide=[{"selector":"th.blank","props":"display:none;"},{"selector":"th.row_heading","props":"display:none;"},{"selector":"td.row_heading","props":"display:none;"}]
        sty = df.style.set_table_styles([{"selector":"table","props":[("border-collapse","collapse"),("width","100%")]},
            {"selector":"th","props":[("background-color",header_bg),("color",header_fg),("font-weight","bold"),("text-align","center"),("padding","6px"),("border",f"1px solid {cell_border_color}")]},
            {"selector":"td","props":[("text-align","center"),("padding","4px"),("border",f"1px solid {cell_border_color}")]}]+css_hide).format(pintar, subset=pd.IndexSlice[:, df.columns])
        display(HTML(sty.to_html(escape=False)))




    # 1.12 GENERACIÓN DE FILAS (Frame estructural y proyectado)

    def icono_status(texto):
        txt = str(texto).lower()
        if "no operar" in txt or "no validada" in txt or "no validado" in txt or "sin sesgo" in txt:
            return "🔴 " + str(texto)
        elif "validada" in txt or "validado" in txt:
            return "🟢 " + str(texto)
        else:
            return str(texto)


    # 1.12.1 Inicialización de listas y ciclo por temporalidades
    valid_tfs  = set()
    filas_estr = []
    filas_proj = []

    for tf in detectar_temporalidades(data):

        # 1.12.2 Carga de variables
        rt      = data.get("Valor_mcdo")
        hi      = data.get(f"Marco_High_{tf}")
        lo      = data.get(f"Marco_Low_{tf}")
        mid     = data.get(f"Mid_Range_{tf}")
        atr     = data.get(f"ATR_{tf}")
        adx     = data.get(f"ADX_{tf}")
        obv     = data.get(f"OBV_Score_{tf}")
        ema     = data.get(f"EMA_Cross_{tf}")
        rsi_raw = data.get(f"RSI_{tf}", None)
        ath     = data.get(f"All_Time_High_{tf}", float("inf"))

        # Flag para Bloque 2: ¿se rompió el ATH?
        data[f"rompió_ATH_{tf}"] = (rt is not None and ath != float("inf") and rt > ath)


        # 1.12.3bis: cálculo de “bias_str”
        categorias = {
            "Tendencia_Alcista":     "Tendencia Alcista",
            "Tendencia_Bajista":     "Tendencia Bajista",
            "Lateral_sesgo_Alcista": "Lateralidad con sesgo Alcista",
            "Lateral_sesgo_Bajista": "Lateralidad con sesgo Bajista",
            "Lateralidad_sin_sesgo": "Lateralidad sin sesgo",
        }
        trend_keys = [
            (k,v) for k,v in data.items()
            if k.startswith("Trend_bias_") and f"_{tf}" in k and isinstance(v,(int,float))
        ]
        if trend_keys:
            ganador, val = max(trend_keys, key=lambda kv:(abs(kv[1]), kv[1]))
            for raw,label in categorias.items():
                if raw in ganador:
                    bias_str = f"{label} {formato_numero(val)}"
                    break
        else:
            bias_str = "Sin sesgo"

        # 1.12.3: posición
        if not all(isinstance(x,(int,float)) for x in (rt,hi,lo,atr,adx,obv,ema)):
            continue
        if   lo + 0.75*atr <= rt <= hi - 0.75*atr:
            pos = "$ in-frame"
        elif rt < lo + 0.75*atr:
            pos = "In-Frame: fuera Low"
        elif hi - 0.75*atr < rt <= hi:
            pos = "In-Frame: fuera High"
        elif hi < rt <= ath:
            pos = "In-Frame: close ATH"
        else:
            pos = "Out-Frame: ATH"

        # 1.12.4 Dirección
        direccion = bias_str or "Sin sesgo"

        # ── Inicializar la flag de frame estructural ──
        is_struct = evaluar_frame_estructural(data, tf)


        # 1.12.5 ──────────── FRAME ESTRUCTURAL ────────────
        # ─────────────── 1.12.5  FRAME ESTRUCTURAL  ───────────────
        #
        #  ▸ Si la ENTRADA NO está validada   →  las 5 columnas
        #      «Tipo Zona operativa / Frame Low / Frame High /
        #       Rango operativo (pts) / ATR (pts)»
        #    se dejan en blanco ("").
        #
        #  ▸ El PD NO interviene en la validación del frame
        #    estructural; aquí solo diagnosticamos dirección.
        # ───────────────────────────────────────────────────────────

        if is_struct:


            # ---------- CASO 1 · Sin sesgo -----------------------------------------
            if direccion.lower().startswith(("sin sesgo", "lateralidad sin sesgo")):
                estado         = "Lateralidad sin sesgo – No operar"
                entrada_valida = False

            # ---------- CASO 2 · Lateralidad con sesgo -----------------------------
            elif "lateralidad con sesgo" in direccion.lower():
                obv_cell  = f_symbolOBV(obv)
                dist_cell = formato_dist_dolar(rt, hi, lo, mid, atr, direccion)
                rsi_cell  = f_symbolRSI(rsi_raw, direccion)

                res_lat = (validar_lateralidad_alcista if "alcista" in direccion.lower()
                          else validar_lateralidad_bajista)(data, tf)

                entrada_valida = res_lat["validado"]
                estado         = res_lat["estado"]

                if entrada_valida:
                    tipo_zona_cell = res_lat.get("tipo_zona", "Zona operativa clásica")
                    f_low_cell     = formato_numero(res_lat["f_low"])
                    f_high_cell    = formato_numero(res_lat["f_high"])
                    rango_cell     = formato_numero(res_lat["rango_pts"])
                    atr_cell       = formato_numero(res_lat["atr"])
                    valid_tfs.add(tf)
                else:                                  #  ❌ NO validada
                    tipo_zona_cell = f_low_cell = f_high_cell = rango_cell = atr_cell = ""

            # ---------- CASO 3 · Tendencia (alcista / bajista) ---------------------
            else:   # incluye la palabra “tendencia”
                es_alc    = "alcista" in direccion.lower()
                adx_cell  = f_symbolADX(adx)
                ema_cell  = f_symbolEMA_estructura(ema)
                obv_cell  = f_symbolOBV(obv)
                dist_cell = formato_dist_dolar(rt, hi, lo, mid, atr, direccion)
                rsi_cell  = "-"

                if adx < umbral_adx_estructura:
                    entrada_valida  = False
                    estado          = "Tendencia débil – No operar"
                else:
                    ok_sig = sum([
                        adx >= umbral_adx_estructura,
                        (ema > 0 and es_alc) or (ema < 0 and not es_alc),
                        (obv == 1 and es_alc) or (obv == -1 and not es_alc)
                    ])
                    entrada_valida = ok_sig >= 2
                    estado = (f"Tendencia {'alcista' if es_alc else 'bajista'} "
                              f"{'validada' if entrada_valida else 'no validada: solo '+str(ok_sig)+'/3 señales – No operar'}")

                if entrada_valida:
                    tipo_zona_cell = "Zona operativa clásica"
                    f_low_cell     = formato_numero(lo)
                    f_high_cell    = formato_numero(hi)
                    rango_cell     = formato_numero(hi - lo)
                    atr_cell       = formato_numero(atr)
                    valid_tfs.add(tf)
                else:                                  #  ❌ NO validada
                    tipo_zona_cell = f_low_cell = f_high_cell = rango_cell = atr_cell = ""

            # ---------- Construcción de la fila ------------------------------------
            filas_estr.append({
                "Activo": data.get("Activo", "ø"),
                "TF": tf,
                "Posición del $": pos,
                "Tipo Frame.": "Frame estructural",
                "Valor mcdo": formato_numero(rt),
                "Dirección": direccion,
                "ADX":  adx_cell  if 'adx_cell'  in locals() else "-",
                "EMA cross": ema_cell if 'ema_cell' in locals() else "-",
                "OBV": obv_cell  if 'obv_cell'  in locals() else "-",
                "Dist. $": dist_cell if 'dist_cell' in locals() else "-",
                "RSI": rsi_cell  if 'rsi_cell'  in locals() else "-",
                "Status final": icono_status(estado),
                "Tipo Zona operativa":   tipo_zona_cell,
                "Frame Low":             f_low_cell,
                "Frame High":            f_high_cell,
                "Rango operativo (pts)": rango_cell,
                "ATR (pts)":             atr_cell
            })

            # Info para etapas posteriores
            data[f"tipo_entrada_{tf}"] = extraer_tipo_entrada(estado)
            data[f"direccion_{tf}"]    = direccion
            data[f"frame_tipo_{tf}"]   = "estructural"
            data[f"frame_direccion_{tf}"] = direccion


            # 1.12.6 ─ Fin de FRAME ESTRUCTURAL

            # 1.12.7 ─ Preparar valid_tfs para FRAME PROYECTADO


        else:



            # 1.12.8 ─────────── FRAME PROYECTADO (solo TF NO válidos en estructural) ───────────
            #     • Ptje Dir (PD) se usa *solo* para fijar la dirección.
            #     • Si la dirección final resulta “Sin sesgo” o bien
            #       el diagnóstico trae “Sin oportunidad proyectiva…”
            #       las cinco columnas de zona/frames/rango/ATR quedan vacías.


            result = determinar_frame_proyectado(data, tf)         # ← func. de la sección 1.10.4


            # —— PD → Dirección final (solo para dirección, no valida entradas) ——
            obv_score =  1 if obv >  0 else (-1 if obv <  0 else 0)
            ema_score =  0.5 if ema >  0 else (-0.5 if ema <  0 else 0)
            adx_score =  1   if adx > 20 else (0.5 if 15 < adx <= 20 else 0)
            rsi_score = (0.5 if rsi_raw > 60 else (-0.5 if rsi_raw < 40 else 0)) \
                        if isinstance(rsi_raw, (int, float)) else 0
            pp_score  =  0.5 if result["tipo_zona"] in (
                            "Proyección anticipada",
                            "Proyección zona operativa completa"
                        ) else (-0.5 if result["tipo_zona"].startswith("Proyección") else 0)

            umbral    = 3.5 if atr < 0.4 else (3.0 if atr <= 0.7 else 2.5)
            pd_score  = obv_score + ema_score + adx_score + rsi_score + pp_score
            lim_lat   = umbral - 0.1

            if   pd_score >=  umbral:
                direccion_final = "Tendencia alcista"
            elif pd_score <= -umbral:
                direccion_final = "Tendencia bajista"
            elif 1 < pd_score <  lim_lat:
                direccion_final = "Lateralidad con sesgo alcista"
            elif -lim_lat < pd_score < -1:
                direccion_final = "Lateralidad con sesgo bajista"
            else:
                direccion_final = "Sin sesgo"



            # —— Ajuste de columnas y status según oportunidad ——
            # Para Frame Proyectado: siempre mostramos Zona Operativa
            tipo_zone   = result["tipo_zona"]
            f_low_val   = result["f_low"]
            f_high_val  = result["f_high"]
            rango_val   = result["rango_pts"]
            atr_val     = result["atr"]

            # Status final: si Sin sesgo, mensaje explícito; en otro caso, usamos el diagnóstico
            if direccion_final == "Sin sesgo":
                status_proj = "Dirección sin sesgo – No operar"
            else:
                status_proj = result["diag"]
                valid_tfs.add(tf)
                data[f"tipo_entrada_{tf}"] = extraer_tipo_entrada(status_proj)


            # —— Label de momentum y distancia proyectada ——
            mom_label = "Fuerte" if adx > umbral_adx_momentum else "Débil"
            dist_proj = formato_dist_dolar(rt, hi, lo, mid, atr, direccion_final)

            # —— Construcción de fila para df_proj ——
            filas_proj.append({
                "Activo":                data.get("Activo","∅"),
                "TF":                    tf,
                "Posición del $":        pos,
                "Tipo Frame":            "Frame proyectado",
                "Valor mcdo":            formato_numero(rt),
                "Momentum":              mom_label,
                "Tipo Zona operativa":   tipo_zone,
                "OBV":       f"R: {obv}\nScore:\u00A0{obv_score}",
                "EMA cross": f"R: {ema}\nScore:\u00A0{ema_score}",
                "ADX":       f"R: {adx}\nScore:\u00A0{adx_score}",
                "RSI":       f"R: {rsi_raw}\nScore:\u00A0{rsi_score}",
                "Ptje Proy.":f"R: {pp_score}\nScore:\u00A0{pp_score}",
                "Umbral":    f"{umbral}",
                "Ptje Dir.": f"{pd_score}",
                "Dirección": direccion_final,
                "Dist. $":   dist_proj,
                "Status final":          icono_status(status_proj),
                "Frame Low":             f_low_val,
                "Frame High":            f_high_val,
                "Rango operativo (pts)": rango_val,
                "ATR (pts)":             atr_val
            })

            # Info para etapas posteriores
            data[f"tipo_entrada_{tf}"]    = extraer_tipo_entrada(status_proj)
            data[f"direccion_{tf}"]       = direccion_final
            data[f"frame_tipo_{tf}"]      = "proyectado"
            data[f"frame_direccion_{tf}"] = direccion_final

            # 1.12.9 Validez final de frame para cada TF
            data[f"frame_valido_{tf}"] = tf in valid_tfs


    # ─────────────────────────────────────────────────────────────────────────────



    # ==== REGISTRO UNIFICADO DE DIRECCIÓN / TIPO FRAME / TIPO ENTRADA / VALIDEZ ====
    # Este bloque asegura que TODAS las TF quedan correctamente etiquetadas
    # con sus campos, sean o no validadas.

    # 1. Dirección final (frame_direccion)
    if f"frame_direccion_{tf}" not in data:
        data[f"frame_direccion_{tf}"] = direccion

    # 2. Tipo de frame (frame_tipo)
    if f"frame_tipo_{tf}" not in data:
        # Si es frame estructural, marca como "estructural", si no, "proyectado"
        data[f"frame_tipo_{tf}"] = "estructural" if is_struct else "proyectado"

    # 3. Tipo de entrada (tipo_entrada)
    if f"tipo_entrada_{tf}" not in data:
        # Usa función auxiliar si tienes estado o pon vacío si no corresponde
        data[f"tipo_entrada_{tf}"] = extraer_tipo_entrada(estado) if 'estado' in locals() else ""

    # 4. Validez final del frame (frame_valido)
    # True si fue validada y está en valid_tfs, False si no
    data[f"frame_valido_{tf}"] = tf in valid_tfs





    # 1.13. DataFrames de resultados ==================================================
    cols_estr = [
        "Activo","TF","Posición del $","Tipo Frame.","Valor mcdo","Dirección",
        "ADX","EMA cross","OBV","Dist. $","RSI","Status final",
        "Tipo Zona operativa","Frame Low","Frame High","Rango operativo (pts)","ATR (pts)"
    ]

    cols_proj = [
        "Activo","TF","Posición del $","Tipo Frame","Valor mcdo","Momentum",
        "Tipo Zona operativa","OBV","EMA cross","ADX","RSI","Ptje Proy.",
        "Umbral","Ptje Dir.","Dirección","Dist. $","Status final",
        "Frame Low","Frame High","Rango operativo (pts)","ATR (pts)"
    ]

    # 1.13.1 Construcción de DataFrames estructurales y proyectados
    df_estr = pd.DataFrame(filas_estr)[cols_estr] if filas_estr else pd.DataFrame(columns=cols_estr)
    df_proj = pd.DataFrame(filas_proj)[cols_proj] if filas_proj else pd.DataFrame(columns=cols_proj)

    from IPython.core.display import HTML, display

    # 1.13.2 Visualización de resultados en HTML (Frame estructural)
    display(HTML("<h2 style='color:#1f497d;'>Resultados análisis marco operación (Bloque 1) - Frame Estructural</h2>"))
    if not df_estr.empty:
        mostrar_tabla(df_estr,
                      header_bg="#1f497d",
                      header_fg="#fff",
                      cell_border_color="#444")
    else:
        display(HTML("<p>No se generaron filas para Frame Estructural</p>"))


    # 1.13.3 Visualización de resultados en HTML (Frame proyectado)
    if not df_proj.empty:
        display(HTML("<h2 style='color:#1f497d;'>Resultados análisis marco operación (Bloque 1) – Frame Proyectado</h2>"))
        mostrar_tabla(
            df_proj,
            header_bg="#f0f8ff",
            header_fg="#1f497d",
            cell_border_color="#1f497d"
        )

    # 1.13.4 Línea final divisoria visual
    display(HTML("<hr style='border:2px solid #1f497d; margin:12px 0;'>"))

    # 1.13.5 Función de extracción de tipo de entrada
    def extraer_tipo_entrada(status):
        s = status.lower()
        if "entrada agresiva" in s:
            return "agresiva"
        if "entrada táctica" in s or "entrada táctica" in s:
            return "táctica"
        if "entrada conservadora" in s:
            return "conservadora"
        return "táctica"  # valor por defecto si no se detecta nada












    # ===========================
    # 2. SCRIPT BLOQUE 2 – TABLA ÚNICA – VERSIÓN FINAL CON MACD 3-ESTADOS,
    # UMBRAL DINÁMICO Y NORMALIZACIÓN MATIZADA
    # ===========================

    # 2.1 DEFINICIÓN DE COLORES POR TEMPORALIDAD (degradado de grises)
    colores_por_tf = {
        "15M": {"bg": "#2C3E50", "fg": "#ECF0F1", "border": "#34495E"},
        "1H":  {"bg": "#566573", "fg": "#FDFEFE", "border": "#5D6D7E"},
        "4H":  {"bg": "#ABB2B9", "fg": "#000000", "border": "#D5D8DC"},
    }

    # 2.2 IMPORTS Y DEPENDENCIAS
    import pandas as pd
    from IPython.display import HTML

    # 2.3 FUNCIONES AUXILIARES DE CÁLCULO (distancias BOS y CHOCH proyectados)

    def parse_float(val):
        """
        Convierte cualquier input a float, manejando comas decimales y valores nulos o vacíos.
        Devuelve None si no es posible convertir.
        """
        if val in (None, "∅", "", "-", "nan"):
            return None
        try:
            return float(str(val).replace(",", "."))
        except (ValueError, TypeError):
            return None

    # 2.3.1.1 Distancia a nivel BOS proyectado, normalizada por ATR
    def raw_BOS_PROY_DIST(data, tf, direccion_tendencia=None, *_):
        key_alc = f"Nivel_Proy_BOS_Alcista_{tf}"
        key_baj = f"Nivel_Proy_BOS_Bajista_{tf}"
        na = parse_float(data.get(key_alc))
        nb = parse_float(data.get(key_baj))
        vm = parse_float(data.get("Valor_mcdo"))
        atr = parse_float(data.get(f"ATR_{tf}"))
        # Sin niveles → excluido
        if na is None and nb is None:
            return {"distancia": None, "direccion_nivel": None, "direccion_tendencia": direccion_tendencia, "caso": None}
        # Decide nivel según sesgo
        if direccion_tendencia:
            dt = direccion_tendencia.lower()
            if "alcista" in dt and na is not None:
                nivel, dn = na, "alcista"
            elif "bajista" in dt and nb is not None:
                nivel, dn = nb, "bajista"
            else:
                nivel, dn = (na, "alcista") if na is not None else (nb, "bajista")
        else:
            nivel, dn = (na, "alcista") if na is not None else (nb, "bajista")
        # Si falta dato clave → excluido
        if nivel is None or vm is None or atr is None or atr == 0:
            return {"distancia": None, "direccion_nivel": dn, "direccion_tendencia": direccion_tendencia, "caso": None}
        distancia = abs(vm - nivel) / atr
        return {"distancia": distancia, "direccion_nivel": dn, "direccion_tendencia": direccion_tendencia, "caso": None}

    # 2.3.1.2 Distancia a nivel CHOCH proyectado, normalizada por ATR
    def raw_CHOCH_PROY_DIST(data, tf, direccion_tendencia=None, *_):
        key_alc = f"Nivel_Proy_CHOCH_Alcista_{tf}"
        key_baj = f"Nivel_Proy_CHOCH_Bajista_{tf}"
        na = parse_float(data.get(key_alc))
        nb = parse_float(data.get(key_baj))
        vm = parse_float(data.get("Valor_mcdo"))
        atr = parse_float(data.get(f"ATR_{tf}"))
        # Sin niveles → excluido
        if na is None and nb is None:
            return {"distancia": None, "direccion_nivel": None, "direccion_tendencia": direccion_tendencia, "caso": None}
        # Decide nivel según sesgo
        if direccion_tendencia:
            dt = direccion_tendencia.lower()
            if "alcista" in dt and na is not None:
                nivel, dn = na, "alcista"
            elif "bajista" in dt and nb is not None:
                nivel, dn = nb, "bajista"
            else:
                nivel, dn = (na, "alcista") if na is not None else (nb, "bajista")
        else:
            nivel, dn = (na, "alcista") if na is not None else (nb, "bajista")
        # Si falta dato clave → excluido
        if nivel is None or vm is None or atr is None or atr == 0:
            return {"distancia": None, "direccion_nivel": dn, "direccion_tendencia": direccion_tendencia, "caso": None}
        distancia = abs(vm - nivel) / atr
        return {"distancia": distancia, "direccion_nivel": dn, "direccion_tendencia": direccion_tendencia, "caso": None}



    # 2.3.2 Validación por nivel de Fibo – extracción dinámica desde los datos
    def get_fibo_val(level: str, tf: str, data: dict) -> float | None:
        """
        level: "38.2%", "50.0%" o "61.8%".
        Busca en `data` todas las variantes posibles (con y sin "(dentro)"),
        devolviendo el valor numérico si lo encuentra, o None si no.
        """
        clave = level.replace(".", "").replace("%", "")
        posibles = [
            f"Fibo_{clave}%_fractal_real_{tf}",
            f"Fibo_{clave}%_fractal_real_{tf} (dentro)",
            f"Fibo_{clave}%_{tf}",
            f"Fibo_{clave}%_{tf} (dentro)",
        ]
        for key in posibles:
            val = data.get(key, None)
            if val not in (None, "∅", ""):
                try:
                    return float(str(val).replace(",", "."))
                except (TypeError, ValueError):
                    return None
        return None




    # 2.4 DEFINICIÓN DE UMBRALES PARA DISTANCIAS LINEALES (dinámico por TF)
    UMBRALES_DIST = {
        'BOS_PROY_DIST': {
            '15M': {'d0': 0.40, 'd1': 0.15},
            '1H':  {'d0': 0.50, 'd1': 0.20},
            '4H':  {'d0': 0.60, 'd1': 0.25},
        },
        'CHOCH_PROY_DIST': {
            '15M': {'d0': 0.40, 'd1': 0.15},
            '1H':  {'d0': 0.50, 'd1': 0.20},
            '4H':  {'d0': 0.60, 'd1': 0.25},
        }
    }


    # 2.5 FUNCIÓN get_norm ── Escala específica por indicador
        # ——— interpolación lineal simple ———
    def interp_lin(x, x0, x1, y0, y1):
        if x <= x0:  return y0
        if x >= x1:  return y1
        if x1 == x0: return (y0 + y1) / 2      # evita división 0
        return y0 + (x - x0) * (y1 - y0) / (x1 - x0)


    def get_norm(indicador, raw_vals, subcaso="", tf=""):
        ind = indicador.upper()
        s   = subcaso.upper()

        # ───── Parseo de clave para usar en condiciones ─────
        es_proyectado = s.startswith("PROYECTADO")
        es_estructural = s.startswith("ESTRUCTURAL")
        es_lateral_alcista = "LATERAL-ALCISTA" in s
        es_lateral_bajista = "LATERAL-BAJISTA" in s
        es_tendencia_alcista = "-ALCISTA-" in s and not es_lateral_alcista
        es_tendencia_bajista = "-BAJISTA-" in s and not es_lateral_bajista
        es_agresiva = "AGRESIVA" in s
        es_tactica = "TÁCTICA" in s
        es_conservadora = "CONSERVADORA" in s
        rompio_ath = "ROMPIO_ATH" in s


        # salvaguarda
        raw  = raw_vals
        norm = 0.0

        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # RSI
        if ind == "RSI":
            raw = float(raw_vals)

            # --- Frame Estructural ---

            # Táctica - Alcista
            if es_estructural and es_tendencia_alcista and es_tactica:
                if raw >= 60:
                    norm = 1.0
                elif 52 <= raw < 60:
                    norm = interp_lin(raw, 52, 60, 0.5, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Alcista
            elif es_estructural and es_tendencia_alcista and es_agresiva:
                if raw >= 60:
                    norm = 1.0
                elif 52 <= raw < 60:
                    norm = interp_lin(raw, 52, 60, 0.5, 1.0)
                else:
                    norm = 0.0

            # Táctica - Bajista
            elif es_estructural and es_tendencia_bajista and es_tactica:
                if raw <= 40:
                    norm = 1.0
                elif 40 < raw <= 47:
                    norm = interp_lin(raw, 47, 40, 0.0, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Bajista
            elif es_estructural and es_tendencia_bajista and es_agresiva:
                if raw <= 40:
                    norm = 1.0
                elif 40 < raw <= 47:
                    norm = interp_lin(raw, 47, 40, 0.0, 1.0)
                else:
                    norm = 0.0

            # Táctica - Lateral Alcista
            elif es_estructural and es_lateral_alcista and es_tactica:
                if raw >= 55:
                    norm = 1.0
                elif 48 <= raw < 55:
                    norm = interp_lin(raw, 48, 55, 0.5, 1.0)
                else:
                    norm = 0.0

            # Táctica - Lateral Bajista
            elif es_estructural and es_lateral_bajista and es_tactica:
                if raw <= 45:
                    norm = 1.0
                elif 45 < raw <= 52:
                    norm = interp_lin(raw, 52, 45, 0.5, 1.0)
                else:
                    norm = 0.0

            # --- Frame Proyectado ---

            # Alcista (Táctica, Agresiva, Conservadora) - No rompió ATH
            elif es_proyectado and es_tendencia_alcista and not rompio_ath:
                if raw >= 60:
                    norm = 1.0
                elif 52 <= raw < 60:
                    norm = interp_lin(raw, 52, 60, 0.5, 1.0)
                else:
                    norm = 0.0

            # Alcista (Táctica, Agresiva) - Sí rompió ATH
            elif es_proyectado and es_tendencia_alcista and rompio_ath and (es_tactica or es_agresiva):
                if raw >= 60:
                    norm = 1.0
                elif 52 <= raw < 60:
                    norm = interp_lin(raw, 52, 60, 0.5, 1.0)
                else:
                    norm = 0.0

            # Lateral Alcista - No rompió ATH (Táctica, Agresiva)
            elif es_proyectado and es_lateral_alcista and not rompio_ath and (es_tactica or es_agresiva):
                if raw >= 55:
                    norm = 1.0
                elif 48 <= raw < 55:
                    norm = interp_lin(raw, 48, 55, 0.5, 1.0)
                else:
                    norm = 0.0

            # Lateral Alcista - No rompió ATH (Conservadora)
            elif es_proyectado and es_lateral_alcista and not rompio_ath and es_conservadora:
                if raw >= 50:
                    norm = 1.0
                elif 45 <= raw < 50:
                    norm = interp_lin(raw, 45, 50, 0.5, 1.0)
                else:
                    norm = 0.0

            # Lateral Alcista - Sí rompió ATH (Táctica, Agresiva)
            elif es_proyectado and es_lateral_alcista and rompio_ath and (es_tactica or es_agresiva):
                if raw <= 55:
                    norm = 1.0
                elif 48 <= raw < 55:
                    norm = interp_lin(raw, 48, 55, 0.5, 1.0)
                else:
                    norm = 0.0

            # Bajista - Conservadora (Frame Proyectado)
            elif es_proyectado and es_tendencia_bajista and es_conservadora:
                if raw <= 40:
                    norm = 1.0
                elif 40 < raw <= 47:
                    norm = interp_lin(raw, 47, 40, 0.0, 1.0)
                else:
                    norm = 0.0

            # Lateral Bajista - Conservadora (Frame Proyectado)
            elif es_proyectado and es_lateral_bajista and es_conservadora:
                if raw <= 45:
                    norm = 1.0
                elif 45 < raw <= 52:
                    norm = interp_lin(raw, 52, 45, 0.5, 1.0)
                else:
                    norm = 0.0

            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # ADX
        elif ind == "ADX":
            raw = float(raw_vals)

            # --- Frame Estructural ---

            # Táctica - Alcista
            if es_estructural and es_tendencia_alcista and es_tactica:
                if raw >= 25:
                    norm = 1.0
                elif 20 <= raw < 25:
                    norm = interp_lin(raw, 20, 25, 0.5, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Alcista
            elif es_estructural and es_tendencia_alcista and es_agresiva:
                if raw >= 25:
                    norm = 1.0
                elif 20 <= raw < 25:
                    norm = interp_lin(raw, 20, 25, 0.5, 1.0)
                else:
                    norm = 0.0

            # Táctica - Bajista
            elif es_estructural and es_tendencia_bajista and es_tactica:
                if raw >= 25:
                    norm = 1.0
                elif 20 <= raw < 25:
                    norm = interp_lin(raw, 20, 25, 0.5, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Bajista
            elif es_estructural and es_tendencia_bajista and es_agresiva:
                if raw >= 25:
                    norm = 1.0
                elif 20 <= raw < 25:
                    norm = interp_lin(raw, 20, 25, 0.5, 1.0)
                else:
                    norm = 0.0

            # --- Frame Proyectado ---

            # Táctica - Alcista - NO ATH
            elif es_proyectado and es_tendencia_alcista and es_tactica and not rompio_ath:
                if raw >= 24:
                    norm = 1.0
                elif 20 <= raw < 24:
                    norm = interp_lin(raw, 20, 24, 0.5, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Alcista - NO ATH
            elif es_proyectado and es_tendencia_alcista and es_agresiva and not rompio_ath:
                if raw >= 25:
                    norm = 1.0
                elif 20 <= raw < 25:
                    norm = interp_lin(raw, 20, 25, 0.5, 1.0)
                else:
                    norm = 0.0

            # Conservadora - Alcista - NO ATH
            elif es_proyectado and es_tendencia_alcista and es_conservadora and not rompio_ath:
                if raw >= 22:
                    norm = 1.0
                elif 18 <= raw < 22:
                    norm = interp_lin(raw, 18, 22, 0.5, 1.0)
                else:
                    norm = 0.0

            # Táctica - Alcista - ROMPIO ATH
            elif es_proyectado and es_tendencia_alcista and es_tactica and rompio_ath:
                if raw >= 26:
                    norm = 1.0
                elif 22 <= raw < 26:
                    norm = interp_lin(raw, 22, 26, 0.5, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Alcista - ROMPIO ATH
            elif es_proyectado and es_tendencia_alcista and es_agresiva and rompio_ath:
                if raw > 27:
                    norm = 1.0
                elif 23 <= raw <= 27:
                    norm = interp_lin(raw, 23, 27, 0.5, 1.0)
                else:
                    norm = 0.0

            # Táctica - Lateral Alcista - NO ATH
            elif es_proyectado and es_lateral_alcista and es_tactica and not rompio_ath:
                if raw >= 16:
                    norm = 1.0
                elif 13 <= raw < 16:
                    norm = interp_lin(raw, 13, 16, 0.5, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Lateral Alcista - NO ATH
            elif es_proyectado and es_lateral_alcista and es_agresiva and not rompio_ath:
                if raw >= 20:
                    norm = 1.0
                elif 15 <= raw < 20:
                    norm = interp_lin(raw, 15, 20, 0.5, 1.0)
                else:
                    norm = 0.0

            # Conservadora - Lateral Alcista - NO ATH
            elif es_proyectado and es_lateral_alcista and es_conservadora and not rompio_ath:
                if raw >= 14:
                    norm = 1.0
                elif 11 <= raw < 14:
                    norm = interp_lin(raw, 11, 14, 0.5, 1.0)
                else:
                    norm = 0.0

            # Táctica - Lateral Alcista - ROMPIO ATH
            elif es_proyectado and es_lateral_alcista and es_tactica and rompio_ath:
                if raw >= 20:
                    norm = 1.0
                elif 17 <= raw < 20:
                    norm = interp_lin(raw, 17, 20, 0.5, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Lateral Alcista - ROMPIO ATH
            elif es_proyectado and es_lateral_alcista and es_agresiva and rompio_ath:
                if raw >= 20:
                    norm = 1.0
                elif 17 <= raw < 20:
                    norm = interp_lin(raw, 17, 20, 0.5, 1.0)
                else:
                    norm = 0.0

            # Conservadora - Bajista (Frame Proyectado)
            elif es_proyectado and es_tendencia_bajista and es_conservadora:
                if raw >= 23:
                    norm = 1.0
                elif 19 <= raw < 23:
                    norm = interp_lin(raw, 19, 23, 0.5, 1.0)
                else:
                    norm = 0.0

            # Conservadora - Lateral Bajista (Frame Proyectado)
            elif es_proyectado and es_lateral_bajista and es_conservadora:
                if raw >= 18:
                    norm = 1.0
                elif 14 <= raw < 18:
                    norm = interp_lin(raw, 14, 18, 0.5, 1.0)
                else:
                    norm = 0.0

            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # OBV_SCORE_RAW
        elif ind == "OBV_SCORE_RAW":
            raw = float(raw_vals)

            # --- Frame Estructural ---

            # Táctica - Alcista
            if es_estructural and es_tendencia_alcista and es_tactica:
                if raw >= 10:
                    norm = 1.0
                elif 0 <= raw < 10:
                    norm = interp_lin(raw, 0, 10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Alcista
            elif es_estructural and es_tendencia_alcista and es_agresiva:
                if raw >= 10:
                    norm = 1.0
                elif 0 <= raw < 10:
                    norm = interp_lin(raw, 0, 10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Táctica - Bajista
            elif es_estructural and es_tendencia_bajista and es_tactica:
                if raw <= -10:
                    norm = 1.0
                elif -10 < raw <= 0.0:
                    norm = interp_lin(raw, 0.0, -10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Agresiva - Bajista
            elif es_estructural and es_tendencia_bajista and es_agresiva:
                if raw <= -10:
                    norm = 1.0
                elif -10 < raw <= 0.0:
                    norm = interp_lin(raw, 0.0, -10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Táctica - Lateral Alcista
            elif es_estructural and es_lateral_alcista and es_tactica:
                if raw >= 10:
                    norm = 1.0
                elif 0 <= raw < 10:
                    norm = interp_lin(raw, 0, 10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Táctica - Lateral Bajista
            elif es_estructural and es_lateral_bajista and es_tactica:
                if raw >= 10:
                    norm = 1.0
                elif 0 <= raw < 10:
                    norm = interp_lin(raw, 0, -10, 0.0, 1.0)
                else:
                    norm = 0.0

            # --- Frame Proyectado ---

            # Alcista (Táctica, Agresiva, Conservadora) - No rompió ATH
            elif es_proyectado and es_tendencia_alcista and not rompio_ath:
                if raw >= 10:
                    norm = 1.0
                elif 0 <= raw < 10:
                    norm = interp_lin(raw, 0, 10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Alcista (Táctica, Agresiva) - Sí rompió ATH
            elif es_proyectado and es_tendencia_alcista and rompio_ath and (es_tactica or es_agresiva):
                if raw >= 10:
                    norm = 1.0
                elif 0 <= raw < 10:
                    norm = interp_lin(raw, 0, 10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Lateral Alcista - No rompió ATH (Táctica, Agresiva)
            elif es_proyectado and es_lateral_alcista and not rompio_ath and (es_tactica or es_agresiva):

                if raw >= 10:
                    norm = 1.0
                elif 0 <= raw < 10:
                    norm = interp_lin(raw, 0, 10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Lateral Alcista - No rompió ATH (Conservadora)
            elif es_proyectado and es_lateral_alcista and not rompio_ath and es_conservadora:
                if raw >= 5:
                    norm = 1.0
                elif 5 < raw <= 10:
                    norm = interp_lin(raw, 5, 10, 1.0, 0.0)
                else:
                    norm = 0.0

            # Lateral Alcista - Sí rompió ATH (Táctica, Agresiva)
            elif es_proyectado and es_lateral_alcista and rompio_ath and (es_tactica or es_agresiva):
                if raw <= -10:
                    norm = 1.0
                elif -10 < raw <= 0.0:
                    norm = interp_lin(raw, 0.0, -10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Bajista - Conservadora (Frame Proyectado)
            elif es_proyectado and es_tendencia_bajista and es_conservadora:
                if raw <= -10:
                    norm = 1.0
                elif -10 < raw <= 0.0:
                    norm = interp_lin(raw, 0.0, -10, 0.0, 1.0)
                else:
                    norm = 0.0

            # Lateral Bajista - Conservadora (Frame Proyectado)
            elif es_proyectado and es_lateral_bajista and es_conservadora:
                if raw <= -8:
                    norm = 1.0
                elif -8 < raw <= 0.0:
                    norm = interp_lin(raw, 0.0, -8, 0.0, 1.0)
                else:
                    norm = 0.0

            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # 2.5 Normalización BOS_PROY_DIST y CHOCH_PROY_DIST
        # ───────────────────────────────────────────────────────────

        # BOS_PROY_DIST
        elif ind == "BOS_PROY_DIST":
            if not isinstance(raw_vals, dict):
                return None, None
            dist = raw_vals.get("distancia")
            atr  = raw_vals.get("atr", 0)
            if dist is None or atr is None:
                return None, None

            dir_bos   = str(raw_vals.get("direccion_nivel","")).lower()
            dir_frame = str(raw_vals.get("direccion_tendencia","")).lower()
            threshold = 0.15 * atr  # <-- umbral en precio

            # Caso direcciones coincidentes
            if dir_bos and dir_frame and (
               dir_bos == dir_frame or
               (dir_bos=="alcista" and "alcista" in dir_frame) or
               (dir_bos=="bajista" and "bajista" in dir_frame)
            ):
                norm = 1.0 if dist <= threshold else 0.5

            # Caso direcciones contrarias
            elif dir_bos and dir_frame:
                norm = -0.3 if dist <= threshold else 0.0


            return dist, norm



        # CHOCH_PROY_DIST
        elif ind == "CHOCH_PROY_DIST":
            if not isinstance(raw_vals, dict):
                return None, None
            dist = raw_vals.get("distancia")
            atr  = raw_vals.get("atr", 0)
            if dist is None or atr is None:
                return None, None

            dir_choch = str(raw_vals.get("direccion_nivel","")).lower()
            dir_frame = str(raw_vals.get("direccion_tendencia","")).lower()
            threshold = 0.15 * atr

            # Caso direcciones coincidentes
            if dir_choch and dir_frame and (
               dir_choch == dir_frame or
               (dir_choch=="alcista" and "alcista" in dir_frame) or
               (dir_choch=="bajista" and "bajista" in dir_frame)
            ):
                norm = 1.0 if dist <= threshold else 0.5

            # Caso direcciones contrarias
            elif dir_choch and dir_frame:
                norm = -0.3 if dist <= threshold else 0.0

            # Sin direcciones (pero con distancia)
            else:
                norm = 1.0 if dist <= threshold else 0.5


            return dist, norm





        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # BBWIDTH (ajustada por TF y sesgo)
        elif ind == "BBWIDTH":
            raw = float(raw_vals)
            tf = tf.upper().replace("TF", "").replace(" ", "")  # Normaliza tf a 15M, 1H, 4H

            # Frame Estructural – Agresiva – Alcista
            if es_estructural and es_tendencia_alcista and es_agresiva:
                if tf == "15M":
                    if 0.007 <= raw <= 0.009:
                        norm = 1.0
                    elif (0.004 <= raw < 0.007) or (0.009 < raw <= 0.011):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.009 <= raw <= 0.011:
                        norm = 1.0
                    elif (0.008 <= raw < 0.009) or (0.011 < raw <= 0.013):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.011 <= raw <= 0.013:
                        norm = 1.0
                    elif (0.009 <= raw < 0.011) or (0.013 < raw <= 0.015):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Estructural – Agresiva – Bajista
            elif es_estructural and es_tendencia_bajista and es_agresiva:
                if tf == "15M":
                    if 0.006 <= raw <= 0.010:
                        norm = 1.0
                    elif (0.004 <= raw < 0.006) or (0.010 < raw <= 0.012):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.008 <= raw <= 0.012:
                        norm = 1.0
                    elif (0.007 <= raw < 0.008) or (0.012 < raw <= 0.014):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.010 <= raw <= 0.014:
                        norm = 1.0
                    elif (0.008 <= raw < 0.010) or (0.014 < raw <= 0.016):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Estructural – Táctica – Lateral Alcista
            elif es_estructural and es_lateral_alcista and es_tactica:
                if 0.004 <= raw <= 0.007:
                    norm = 1.0
                elif (0.002 <= raw < 0.004) or (0.007 < raw <= 0.010):
                    norm = 0.5
                else:
                    norm = 0.0

            # Frame Estructural – Táctica – Lateral Bajista
            elif es_estructural and es_lateral_bajista and es_tactica:
                if 0.005 <= raw <= 0.008:
                    norm = 1.0
                elif (0.003 <= raw < 0.005) or (0.008 < raw <= 0.010):
                    norm = 0.5
                else:
                    norm = 0.0

            # Frame Proyectado – Agresiva – Alcista – No rompió ATH
            elif es_proyectado and es_tendencia_alcista and es_agresiva and not rompio_ath:
                if tf == "15M":
                    if 0.007 <= raw <= 0.009:
                        norm = 1.0
                    elif (0.004 <= raw < 0.007) or (0.009 < raw <= 0.011):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.009 <= raw <= 0.011:
                        norm = 1.0
                    elif (0.008 <= raw < 0.009) or (0.011 < raw <= 0.013):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.011 <= raw <= 0.013:
                        norm = 1.0
                    elif (0.009 <= raw < 0.011) or (0.013 < raw <= 0.015):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Proyectado – Táctica – Alcista – Sí rompió ATH
            elif es_proyectado and es_tendencia_alcista and es_tactica and rompio_ath:
                if tf == "15M":
                    if 0.006 <= raw <= 0.008:
                        norm = 1.0
                    elif (0.003 <= raw < 0.006) or (0.008 < raw <= 0.010):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.008 <= raw <= 0.010:
                        norm = 1.0
                    elif (0.007 <= raw < 0.008) or (0.010 < raw <= 0.012):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.010 <= raw <= 0.012:
                        norm = 1.0
                    elif (0.008 <= raw < 0.010) or (0.012 < raw <= 0.014):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Proyectado – Agresiva – Alcista – Sí rompió ATH
            elif es_proyectado and es_tendencia_alcista and es_agresiva and rompio_ath:
                if tf == "15M":
                    if 0.007 <= raw <= 0.009:
                        norm = 1.0
                    elif (0.004 <= raw < 0.007) or (0.009 < raw <= 0.011):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.009 <= raw <= 0.011:
                        norm = 1.0
                    elif (0.008 <= raw < 0.009) or (0.011 < raw <= 0.013):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.011 <= raw <= 0.013:
                        norm = 1.0
                    elif (0.009 <= raw < 0.011) or (0.013 < raw <= 0.015):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Proyectado – Agresiva – Lateral Alcista – No rompió ATH
            elif es_proyectado and es_lateral_alcista and es_agresiva and not rompio_ath:
                if tf == "15M":
                    if 0.006 <= raw <= 0.008:
                        norm = 1.0
                    elif (0.003 <= raw < 0.006) or (0.008 < raw <= 0.010):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.008 <= raw <= 0.010:
                        norm = 1.0
                    elif (0.007 <= raw < 0.008) or (0.010 < raw <= 0.012):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.010 <= raw <= 0.012:
                        norm = 1.0
                    elif (0.008 <= raw < 0.010) or (0.012 < raw <= 0.014):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Proyectado – Táctica – Lateral Alcista – Sí rompió ATH
            elif es_proyectado and es_lateral_alcista and es_tactica and rompio_ath:
                if tf == "15M":
                    if 0.006 <= raw <= 0.008:
                        norm = 1.0
                    elif (0.003 <= raw < 0.006) or (0.008 < raw <= 0.010):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.008 <= raw <= 0.010:
                        norm = 1.0
                    elif (0.007 <= raw < 0.008) or (0.010 < raw <= 0.012):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.010 <= raw <= 0.012:
                        norm = 1.0
                    elif (0.008 <= raw < 0.010) or (0.012 < raw <= 0.014):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Proyectado – Agresiva – Lateral Alcista – Sí rompió ATH
            elif es_proyectado and es_lateral_alcista and es_agresiva and rompio_ath:
                if tf == "15M":
                    if 0.006 <= raw <= 0.008:
                        norm = 1.0
                    elif (0.003 <= raw < 0.006) or (0.008 < raw <= 0.010):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.008 <= raw <= 0.010:
                        norm = 1.0
                    elif (0.007 <= raw < 0.008) or (0.010 < raw <= 0.012):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.010 <= raw <= 0.012:
                        norm = 1.0
                    elif (0.008 <= raw < 0.010) or (0.012 < raw <= 0.014):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Proyectado – Conservadora – Bajista
            elif es_proyectado and es_tendencia_bajista and es_conservadora:
                if tf == "15M":
                    if 0.006 <= raw <= 0.008:
                        norm = 1.0
                    elif (0.003 <= raw < 0.006) or (0.008 < raw <= 0.010):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.008 <= raw <= 0.010:
                        norm = 1.0
                    elif (0.007 <= raw < 0.008) or (0.010 < raw <= 0.012):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.010 <= raw <= 0.012:
                        norm = 1.0
                    elif (0.008 <= raw < 0.010) or (0.012 < raw <= 0.014):
                        norm = 0.5
                    else:
                        norm = 0.0

            # Frame Proyectado – Conservadora – Lateral Bajista
            elif es_proyectado and es_lateral_bajista and es_conservadora:
                if tf == "15M":
                    if 0.005 <= raw <= 0.008:
                        norm = 1.0
                    elif (0.003 <= raw < 0.005) or (0.008 < raw <= 0.010):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "1H":
                    if 0.007 <= raw <= 0.010:
                        norm = 1.0
                    elif (0.005 <= raw < 0.007) or (0.010 < raw <= 0.012):
                        norm = 0.5
                    else:
                        norm = 0.0
                elif tf == "4H":
                    if 0.009 <= raw <= 0.012:
                        norm = 1.0
                    elif (0.007 <= raw < 0.009) or (0.012 < raw <= 0.014):
                        norm = 0.5
                    else:
                        norm = 0.0

            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # FIBO (Frame Estructural – retroceso 38–61 %)
        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # FIBO (Frame Estructural – retroceso 38–61 %)
        elif ind == "FIBO":
            # extraemos del dict raw_vals
            if isinstance(raw_vals, dict):
                fractal_dir = str(raw_vals.get("fractal_dir","")).lower()
                dir_frame   = str(raw_vals.get("dir_frame", "" )).lower()
                dentro      = bool(raw_vals.get("dentro_rango",False))
                raw         = float(raw_vals.get("fibo_val",0))
            else:
                fractal_dir = dir_frame = ""
                dentro      = False
                raw         = float(raw_vals or 0)
            # la lógica 1 / 0.5 / -0.3
            if fractal_dir == dir_frame and fractal_dir in ("alcista","bajista"):
                norm = 1.0 if dentro else 0.5
            else:
                norm = -0.3
            return raw, norm

        # ───────────────────────────────────────────────────────────
        # FIBO_ENV (Frame Proyectado – zona envolvente 78.6–1.00)
        elif ind == "FIBO_ENV":
            if isinstance(raw_vals, dict):
                fractal_dir = str(raw_vals.get("fractal_dir","")).lower()
                dir_frame   = str(raw_vals.get("dir_frame", "" )).lower()
                dentro      = bool(raw_vals.get("dentro_rango",False))
                raw         = float(raw_vals.get("fibo_val",0))
            else:
                fractal_dir = dir_frame = ""
                dentro      = False
                raw         = float(raw_vals or 0)
            # misma regla de pesos que FIBO
            if fractal_dir == dir_frame and fractal_dir in ("alcista","bajista"):
                norm = 1.0 if dentro else 0.5
            else:
                norm = -0.3
            return raw, norm







        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # DIST_ATH_LE_ATR
        elif ind == "DIST_ATH_LE_ATR":
            raw = float(raw_vals)
            # Conservadora, Frame Proyectado, Alcista o Lateral Alcista
            if es_proyectado and es_conservadora and (es_tendencia_alcista or es_lateral_alcista):
                if raw <= 1.0:
                    norm = 1.0
                elif 1.0 < raw <= 2.0:
                    norm = 0.5
                else:
                    norm = 0.0
            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # DIST_HTFHIGH_10PCT
        elif ind == "DIST_HTFHIGH_10PCT":
            raw = float(raw_vals)
            # Conservadora, Frame Proyectado, Alcista o Lateral Alcista
            if es_proyectado and es_conservadora and (es_tendencia_alcista or es_lateral_alcista):
                if raw >= 0.10:
                    norm = 1.0
                elif 0.05 <= raw < 0.10:
                    norm = 0.5
                else:
                    norm = 0.0
            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # ZONACENTRAL
        elif ind == "ZONACENTRAL":
            raw = float(raw_vals)
            # Lateral Alcista – Frame Estructural – Táctica
            if es_estructural and es_lateral_alcista and es_tactica:
                if raw <= 0.10:
                    norm = 1.0
                elif 0.10 < raw <= 0.20:
                    norm = 0.5
                else:
                    norm = 0.0
            # Lateral Bajista – Frame Estructural – Táctica
            elif es_estructural and es_lateral_bajista and es_tactica:
                if raw <= 0.15:
                    norm = 1.0
                elif 0.15 < raw <= 0.25:
                    norm = 0.5
                else:
                    norm = 0.0
            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # REBOTEEXTREMO
        elif ind == "REBOTEEXTREMO":
            # raw_vals debe ser un dict con los tres valores: RSI, BBW, VolRel
            if isinstance(raw_vals, dict):
                rsi = float(raw_vals.get("rsi", 999))
                bbw = float(raw_vals.get("bbw", 999))
                volrel = float(raw_vals.get("volrel", 0))
            else:
                # Si sólo viene un valor, score siempre 0 (no recomendado)
                rsi = bbw = 999
                volrel = 0

            # Lateral Alcista/Bajista – Frame Estructural – Táctica
            if es_estructural and es_tactica and (es_lateral_alcista or es_lateral_bajista):
                if rsi < 30 and bbw < 0.006 and volrel >= 1:
                    norm = 1.0
                else:
                    norm = 0.0
            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # VOLREL
        elif ind == "VOLREL":
            raw = float(raw_vals)
            # Lateral Alcista – Frame Estructural – Táctica
            if es_estructural and es_lateral_alcista and es_tactica:
                if raw >= 1.2:
                    norm = 1.0
                elif 0.9 <= raw < 1.2:
                    norm = 0.5
                else:
                    norm = 0.0
            # Lateral Bajista – Frame Estructural – Táctica
            elif es_estructural and es_lateral_bajista and es_tactica:
                if raw >= 1.0:
                    norm = 1.0
                elif 0.7 <= raw < 1.0:
                    norm = 0.5
                else:
                    norm = 0.0
            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # DIST_HTFLOW_10PCT
        elif ind == "DIST_HTFLOW_10PCT":
            raw = float(raw_vals)
            # Conservadora, Frame Proyectado, Bajista o Lateral Bajista
            if es_proyectado and es_conservadora and (es_tendencia_bajista or es_lateral_bajista):
                if raw <= 1.0:
                    norm = 1.0
                elif 1.0 < raw <= 2.0:
                    norm = 0.5
                else:
                    norm = 0.0
            else:
                norm = 0.0


        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # MACD (line, signal, hist)
        elif ind == "MACD":
            # Entrada: (line, signal, hist)
            try:
                line, signal, hist = raw_vals
            except (TypeError, ValueError):
                line = signal = hist = 0.0

            # Por defecto
            norm = 0.0

            # === CASOS ALCISTAS (Frame Estructural y Proyectado, todas las entradas)
            if (es_tendencia_alcista or es_lateral_alcista) and (es_estructural or es_proyectado):
                # Caso 1: Line > 0 (alcista) y Line > Signal (impulso creciente)
                if line > 0 and line > signal:
                    norm = 1.0
                # Caso 2: Line < 0 (bajista) y Line > Signal (impulso recuperando)
                elif line < 0 and line > signal:
                    norm = 0.5
                # Caso 3: Line > 0 (alcista) y Line < Signal (impulso decreciente)
                elif line > 0 and line < signal:
                    norm = 0.5
                # Caso 4: Line < 0 (bajista) y Line < Signal (impulso confirmadamente bajista)
                elif line < 0 and line < signal:
                    norm = 0.0

            # === CASOS BAJISTAS (Frame Estructural y Proyectado, todas las entradas)
            elif (es_tendencia_bajista or es_lateral_bajista) and (es_estructural or es_proyectado):
                # Caso 1: Line < 0 (bajista) y Line < Signal (impulso decreciente, caída confirmada)
                if line < 0 and line < signal:
                    norm = 0.0
                # Caso 2: Line < 0 (bajista) y Line > Signal (impulso creciente, recuperación)
                elif line < 0 and line > signal:
                    norm = 0.5
                # Caso 3: Line > 0 (alcista) y Line < Signal (impulso decreciente tras rebote)
                elif line > 0 and line < signal:
                    norm = 0.5
                # Caso 4: Line > 0 (alcista) y Line > Signal (rebote con impulso creciente)
                elif line > 0 and line > signal:
                    norm = 1.0

            # Fallback (por si hay un subcaso futuro no mapeado)
            else:
                norm = 0.0



        # ───────────────────────────────────────────────────────────
        # ───────────────────────────────────────────────────────────
        # Fallback genérico (para indicadores aún no implementados o mal escritos)
        else:
            try:
                raw = float(raw_vals)
            except (TypeError, ValueError):
                raw = 0.0
            norm = 0.0
            print(f"\n❗DEBUG Fallback: Indicador '{ind}' no reconocido en subcaso '{s}' — se devuelve norm = 0.0")

        return raw, norm




    # 2.6 DICCIONARIO MAESTRO DE PESOS DE CALIBRACIÓN – TODOS LOS SUBCASOS

    PESOS_CALIBRACION = {
        # Frame Estructural
        "ESTRUCTURAL-ALCISTA-TÁCTICA": {
            "RSI": 20, "ADX": 25, "OBV_SCORE_RAW": 15, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10,"FIBO": 10
        },
        "ESTRUCTURAL-ALCISTA-AGRESIVA": {
            "RSI": 15, "ADX": 25, "OBV_SCORE_RAW": 20, "BBWIDTH": 10, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },
        "ESTRUCTURAL-BAJISTA-TÁCTICA": {
            "RSI": 20, "ADX": 20, "OBV_SCORE_RAW": 15, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "FIBO": 15,
        },
        "ESTRUCTURAL-BAJISTA-AGRESIVA": {
            "RSI": 20, "ADX": 25, "OBV_SCORE_RAW": 15, "BBWIDTH": 10, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },
        "ESTRUCTURAL-LATERAL-ALCISTA": {
            "RSI": 10, "OBV_SCORE_RAW": 5, "BBWIDTH": 20, "REBOTEEXTREMO": 15, "VOLREL": 15, "ZONACENTRAL": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },
        "ESTRUCTURAL-LATERAL-BAJISTA": {
            "RSI": 10, "OBV_SCORE_RAW": 5, "BBWIDTH": 20, "REBOTEEXTREMO": 15, "VOLREL": 15, "ZONACENTRAL": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },

        # Frame Proyectado – Alcista
        "PROYECTADO-ALCISTA-TÁCTICA-NO_ATH": {
            "RSI": 25, "ADX": 20, "OBV_SCORE_RAW": 15, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "FIBO_ENV": 10
        },
        "PROYECTADO-ALCISTA-AGRESIVA-NO_ATH": {
            "RSI": 20, "ADX": 25, "OBV_SCORE_RAW": 15, "BBWIDTH": 10, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },
        "PROYECTADO-ALCISTA-CONSERVADORA-NO_ATH": {
            "RSI": 15, "ADX": 15, "OBV_SCORE_RAW": 10, "DIST_ATH_LE_ATR": 15, "DIST_HTFHIGH_10PCT": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "FIBO_ENV": 10
        },
        "PROYECTADO-ALCISTA-TÁCTICA-ROMPIO_ATH": {
            "RSI": 25, "ADX": 20, "OBV_SCORE_RAW": 15, "BBWIDTH": 10, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },
        "PROYECTADO-ALCISTA-AGRESIVA-ROMPIO_ATH": {
            "RSI": 20, "ADX": 25, "OBV_SCORE_RAW": 15, "BBWIDTH": 10, "MACD": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },

        # Frame Proyectado – Lateral Alcista
        "PROYECTADO-LATERAL-ALCISTA-TÁCTICA-NO_ATH": {
            "RSI": 20, "ADX": 20, "OBV_SCORE_RAW": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "FIBO_ENV": 10, "MACD": 10
        },
        "PROYECTADO-LATERAL-ALCISTA-AGRESIVA-NO_ATH": {
            "RSI": 20, "ADX": 20, "OBV_SCORE_RAW": 15, "BBWIDTH": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "MACD": 10
        },
        "PROYECTADO-LATERAL-ALCISTA-CONSERVADORA-NO_ATH": {
            "RSI": 15, "ADX": 15, "OBV_SCORE_RAW": 10, "DIST_ATH_LE_ATR": 15, "DIST_HTFHIGH_10PCT": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "FIBO_ENV": 10
        },
        "PROYECTADO-LATERAL-ALCISTA-TÁCTICA-ROMPIO_ATH": {
            "RSI": 15, "ADX": 20, "OBV_SCORE_RAW": 15, "BBWIDTH": 10, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "FIBO_ENV": 10, "MACD": 10
        },
        "PROYECTADO-LATERAL-ALCISTA-AGRESIVA-ROMPIO_ATH": {
            "RSI": 15, "ADX": 25, "OBV_SCORE_RAW": 15, "BBWIDTH": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "MACD": 10
        },

        # Frame Proyectado – Bajista
        "PROYECTADO-BAJISTA-CONSERVADORA": {
            "RSI": 20, "ADX": 25, "OBV_SCORE_RAW": 10, "BBWIDTH": 10, "DIST_HTFLOW_10PCT": 15, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10
        },

        # Frame Proyectado – Lateral Bajista
        "PROYECTADO-LATERAL-BAJISTA-CONSERVADORA": {
            "RSI": 18, "ADX": 22, "OBV_SCORE_RAW": 10, "BBWIDTH": 15, "DIST_HTFLOW_10PCT": 5, "BOS_PROY_DIST": 10, "CHOCH_PROY_DIST": 10, "MACD": 10
        },
    }



    # 2.6bis FUNCIÓN PARA GENERAR CLAVE ESTÁNDAR DEL SUBCASO

    def get_subcaso_key(tipo_frame: str, direccion: str, tipo_entrada: str, rompio_ath: bool) -> str:
        """
        Devuelve la clave normalizada que se usa para buscar en PESOS_CALIBRACION.
        Orden de ifs:
          • Frame Estructural: lateralidad primero (siempre táctica en lateralidad),
            luego tendencia alcista/bajista con táctica o agresiva.
          • Frame Proyectado: lateralidad primero (táctica/agresiva/conservadora + ATH),
            luego tendencia alcista (táctica/agresiva/conservadora + ATH),
            y finalmente tendencia bajista agresiva.
        """
        dir_ = direccion.lower()
        ent = tipo_entrada.upper()
        tf  = tipo_frame.upper()

        # ─────────── FRAME ESTRUCTURAL ───────────
        if tf == "FRAME ESTRUCTURAL":
            # 1) Lateralidad siempre TÁCTICA
            if "lateralidad" in dir_:
                if "alcista" in dir_:
                    return "ESTRUCTURAL-LATERAL-ALCISTA"
                elif "bajista" in dir_:
                    return "ESTRUCTURAL-LATERAL-BAJISTA"

            # 2) Tendencia
            if "alcista" in dir_:
                if ent in ("TÁCTICA","AGRESIVA"):
                    return f"ESTRUCTURAL-ALCISTA-{ent}"
            elif "bajista" in dir_:
                if ent in ("TÁCTICA","AGRESIVA"):
                    return f"ESTRUCTURAL-BAJISTA-{ent}"

        # ─────────── FRAME PROYECTADO ───────────
        elif tf == "FRAME PROYECTADO":
            # 1) Lateralidad
            if "lateralidad" in dir_:
                sesgo = "ALCISTA" if "alcista" in dir_ else "BAJISTA"
                # Para proyectado-lateral-bajista solo existe AGRESIVA
                if sesgo == "BAJISTA":
                    return "PROYECTADO-LATERAL-BAJISTA-AGRESIVA"
                # Para alcista: TÁCTICA, AGRESIVA o CONSERVADORA + ROMPIO/NO_ATH
                if ent in ("TÁCTICA","AGRESIVA"):
                    sat = "ROMPIO_ATH" if (rompio_ath and ent!="CONSERVADORA") else "NO_ATH"
                    return f"PROYECTADO-LATERAL-ALCISTA-{ent}-{sat}"
                else:
                    return "PROYECTADO-LATERAL-ALCISTA-CONSERVADORA-NO_ATH"

            # 2) Tendencia alcista
            if "alcista" in dir_:
                if ent in ("TÁCTICA","AGRESIVA"):
                    sat = "ROMPIO_ATH" if rompio_ath else "NO_ATH"
                    return f"PROYECTADO-ALCISTA-{ent}-{sat}"
                elif ent == "CONSERVADORA":
                    return "PROYECTADO-ALCISTA-CONSERVADORA-NO_ATH"

            # 3) Tendencia bajista (solo agresiva)
            if "bajista" in dir_ and ent == "AGRESIVA":
                return "PROYECTADO-BAJISTA-AGRESIVA"

        # ─────────── DEFAULT ───────────
        return "NO_MATCH"



    # 2.6 FUNCIÓN get_pesos ————
    def get_pesos(subcaso_key: str):
        pesos = PESOS_CALIBRACION.get(subcaso_key, {})
        if not pesos:
            print(f"[ADVERTENCIA] Subcaso técnico no encontrado: {subcaso_key}")
        return pesos




    # 2.7 CÁLCULO DE SCORE TÉCNICO FINAL – Integración de pesos y normalizaciones
    def calcular_score_final(fila, subcaso, tf, data):
        s = subcaso.upper()
        es_lateral_alcista   = "LATERAL-ALCISTA" in s
        es_tendencia_alcista = "-ALCISTA-" in s and not es_lateral_alcista

        indicadores  = fila["indicadores"]
        valores_raw  = fila["valores"]

        # ───── DEBUG DEBUT SCORE ─────
        print(f"\n[DEBUG DEBUT SCORE] TF={tf}  Subcaso={subcaso}")
        for nombre_ind, raw_val in valores_raw.items():
            print(f"   {nombre_ind:20s} → {raw_val!r}")
        print("─" * 40)
        # ─────────────────────────────

        # Obtengo los pesos calibrados
        pesos_orig = get_pesos(subcaso)
        # Normalizo nombres de clave de peso
        pesos = {
            clave.upper().replace(" ", "_").replace("-", "_"): valor
            for clave, valor in pesos_orig.items()
        }

        numerador   = 0.0
        denominador = 0.0
        detalle     = {}

        # ─────────── Bucle de indicadores ───────────
        for ind in indicadores:
            # 1) tomo raw_val
            raw_val = valores_raw.get(ind)
            if raw_val is None or raw_val == "Excluido":
                detalle[ind] = "Excluido"
                continue

            # 2) tomo peso calibrado
            clave_ind = ind.upper().replace(" ", "_").replace("-", "_")
            peso = pesos.get(clave_ind)
            if peso is None:
                detalle[ind] = "Excluido"
                continue

            # 3) normalizo con get_norm (incluye tu lógica de FIBO / FIBO_ENV)
            raw, norm = get_norm(ind, raw_val, subcaso=subcaso, tf=tf)
            if norm is None:
                detalle[ind] = "Excluido"
                continue

            # 4) acumulo numerador y denominador
            contrib = round(norm * peso, 3)
            numerador   += norm * peso
            denominador += abs(peso)
            detalle[ind] = contrib
        # ───────────────────────────────────────────────

        # Si no hay nada que sumar, devuelvo 0
        if denominador == 0:
            return 0.0, detalle

        score_final = numerador / denominador
        return round(score_final, 3), detalle




    # 2.8 Usamos el mismo valid_tfs que construimos en el Bloque 1



    # 2.8bis Función para calcular el umbral dinámico en %
    def get_umbral(subcaso: str, factor: float) -> float:
        """
        Devuelve el umbral dinámico en porcentaje para el subcaso dado
        y el factor de ajuste (ATR).
        - subcaso: cadena con el nombre del escenario (ej. "Táctica en Tendencia alcista ...")
        - factor:  factor de penalización/ajuste según ATR (0.8, 0.9, 1.0)
        """
        # Umbral base por defecto (puedes ajustar esta lógica más adelante)
        umbral_base_pct = 50.0

        # Si el subcaso es proyección conservadora, subir un poco el umbral
        if "conservadora" in subcaso.lower():
            umbral_base_pct += 5.0

        # Retorna el umbral ajustado por el factor
        return umbral_base_pct * factor






    # ════════════════════════════════════════════════════════════════════════
    # 2.9 BUCLE PRINCIPAL ─ Normalización, scoring y tabla técnica por TF
    #     ▸ Usa ‘valid_tfs’ construido en Bloque 1
    #     ▸ Determina la CLAVE de sub-caso → get_pesos → get_norm → score final
    #     ▸ Muestra la tabla “Resultados Análisis Técnico – TF …”
    # ════════════════════════════════════════════════════════════════════════

    import re
    import pandas as pd
    from IPython.display import HTML

    # Si valid_tfs quedó vacío, usa todas las TF detectadas
    if not valid_tfs:
        valid_tfs = set(detectar_temporalidades(data))

    for tf in sorted(valid_tfs):
        # FILTRO: solo procesar TF validadas en Bloque 1
        if not data.get(f"frame_valido_{tf}", False):
            continue

        # ───────────── Datos base ─────────────
        activo     = data.get("Activo", "")
        tipo_ent   = (data.get(f"tipo_entrada_{tf}", "") or "").strip().lower()
        direccion  = (data.get(f"frame_direccion_{tf}", "") or "").lower()
        direccion  = direccion.replace("con sesgo", "sesgo")
        direccion  = re.sub(r"\s*[−-]?\s*\d+[,.]?\d*\s*$", "", direccion).strip()
        frame_tipo = (data.get(f"frame_tipo_{tf}", "") or "").strip().lower()
        rompio_ath = bool(data.get(f"rompió_ATH_{tf}", False))
        tipo_frame = "frame estructural" if frame_tipo == "estructural" else "frame proyectado"

        # ───────────── Selección de CLAVE de subcaso ─────────────

        clave = get_subcaso_key(tipo_frame, direccion, tipo_ent, rompio_ath)

        # 🔒 Corrección de guiones NO ASCII
        clave = clave.replace("–", "-").replace("—", "-").replace("‐", "-")




        # ─────────── Pesos y colección de valores RAW ────────────
        pesos = get_pesos(clave)
        if not pesos:
            continue

        valores_raw = {}

        for ind in pesos:
            if ind == "MACD":
                valores_raw[ind] = (
                    data.get(f"MACD_Line_{tf}", 0.0),
                    data.get(f"MACD_Signal_{tf}", 0.0),
                )

            elif ind == "OBV_SCORE_RAW":
                alt = f"OBV_SCORE_RAW_{tf}"
                valores_raw[ind] = data.get(alt, 0.0)


            # ───────────────────────────────────────────────────────────
            # 2.9.1.1 Extracción de valores RAW para FIBO (Frame Estructural)
            elif ind == "FIBO":
                # 1) Valor de retroceso (38.2%, 50.0% o 61.8%)
                fibo_val = (
                    get_fibo_val("38.2%", tf, data)
                    or get_fibo_val("50.0%", tf, data)
                    or get_fibo_val("61.8%", tf, data)
                )

                # 2) Detecta fractal_dir usando tus columnas exactas
                alc_key = f"Tipo_de_fractal_real_{tf}:_Alcista"
                baj_key = f"Tipo_de_fractal_real_{tf}:_Bajista"
                if data.get(alc_key) not in (None, "∅", ""):
                    fractal_dir = "alcista"
                elif data.get(baj_key) not in (None, "∅", ""):
                    fractal_dir = "bajista"
                else:
                    fractal_dir = None

                # 3) Detecta dir_frame (tendencia vs lateralidad)
                raw_dir = (data.get(f"frame_direccion_{tf}", "") or "").lower()
                if "lateral" in raw_dir:
                    dir_frame = "lateral-alcista" if "alcista" in raw_dir else "lateral-bajista"
                elif "alcista" in raw_dir:
                    dir_frame = "alcista"
                elif "bajista" in raw_dir:
                    dir_frame = "bajista"
                else:
                    dir_frame = None

                # 4) Flag dentro de rango 0.382–0.618
                dentro_rango = False
                if isinstance(fibo_val, float):
                    dentro_rango = 0.382 <= fibo_val <= 0.618

                valores_raw[ind] = {
                    "fibo_val":    fibo_val,
                    "fractal_dir": fractal_dir,
                    "dir_frame":   dir_frame,
                    "dentro_rango": dentro_rango,
                }



            # 2.9.1.2 Extracción de valores RAW para FIBO_ENV

            elif ind == "FIBO_ENV":
                # 1) Valor de Fibo zona envolvente
                fibo_val = parse_float(data.get(f"Fibo_zona_envolvente_{tf}", None))

                # 2) Determina fractal_dir leyendo tus columnas EXACTAS
                alc_key = f"Tipo_de_fractal_real_{tf}:_Alcista"
                baj_key = f"Tipo_de_fractal_real_{tf}:_Bajista"
                if data.get(alc_key) not in (None, "∅", ""):
                    fractal_dir = "alcista"
                elif data.get(baj_key) not in (None, "∅", ""):
                    fractal_dir = "bajista"
                else:
                    fractal_dir = None


                # 3) Determina dir_frame, diferenciando lateral vs tendencia
                raw_dir = (data.get(f"frame_direccion_{tf}", "") or "").lower()

                if "lateral" in raw_dir or "lateralidad" in raw_dir:
                    # Si en tu texto aparece “lateral” → marcamos lateral-alcista/bajista
                    dir_frame = "lateral-alcista" if "alcista" in raw_dir else "lateral-bajista"
                elif "alcista" in raw_dir:
                    dir_frame = "alcista"
                elif "bajista" in raw_dir:
                    dir_frame = "bajista"
                else:
                    dir_frame = None

                # DEBUG rápido (opcional):
                print(f"[DEBUG DIR_FRAME] TF={tf} | raw_dir={raw_dir!r} -> dir_frame={dir_frame}")


                # 4) Flag dentro de rango 0.618–1.00
                dentro_rango = bool(data.get(f"Fibo_dentro_rango_env_{tf}", False))

                valores_raw[ind] = {
                    "fibo_val":     fibo_val,
                    "fractal_dir":  fractal_dir,
                    "dir_frame":    dir_frame,
                    "dentro_rango": dentro_rango,
                }







            # ───────────────────────────────────────────────────────────
            # 2.9.1 Extracción de valores RAW para BOS_PROY_DIST
            # ───────────────────────────────────────────────────────────
            # … dentro del for ind in pesos: …

            elif ind == "BOS_PROY_DIST":
                direccion_tendencia = data.get(f"frame_direccion_{tf}", None) or data.get(f"Direccion_{tf}", None)
                valor_mcdo    = parse_float(data.get("Valor_mcdo"))
                atr           = parse_float(data.get(f"ATR_{tf}"))
                nivel_alc     = parse_float(data.get(f"Nivel_Proy_BOS_Alcista_{tf}"))
                nivel_baj     = parse_float(data.get(f"Nivel_Proy_BOS_Bajista_{tf}"))

                if nivel_alc is None and nivel_baj is None:
                    dist = None
                    direccion_nivel = None
                else:
                    dt = str(direccion_tendencia or "").lower()
                    if "alcista" in dt and nivel_alc is not None:
                        nivel, direccion_nivel = nivel_alc, "alcista"
                    elif "bajista" in dt and nivel_baj is not None:
                        nivel, direccion_nivel = nivel_baj, "bajista"
                    else:
                        nivel = nivel_alc if nivel_alc is not None else nivel_baj
                        direccion_nivel = "alcista" if nivel_alc is not None else "bajista"
                    # *** Aquí tomamos la distancia en precio absoluto ***
                    dist = abs(valor_mcdo - nivel) if (valor_mcdo is not None and nivel is not None) else None

                valores_raw[ind] = {
                    "distancia": dist,
                    "atr":        atr,               # <-- incluimos ATR
                    "direccion_nivel":    direccion_nivel,
                    "direccion_tendencia": direccion_tendencia,
                    "caso": None,
                }


            elif ind == "CHOCH_PROY_DIST":
                direccion_tendencia = data.get(f"frame_direccion_{tf}", None) or data.get(f"Direccion_{tf}", None)
                valor_mcdo    = parse_float(data.get("Valor_mcdo"))
                atr           = parse_float(data.get(f"ATR_{tf}"))
                # fallback de nombres si es necesario
                key_al = f"Nivel_Proy_CHOCH_Alcista_{tf}"
                key_ba = f"Nivel_Proy_CHOCH_Bajista_{tf}"
                if key_al not in data and f"Nivel_Proy_CHoCH_Alcista_{tf}" in data:
                    key_al = f"Nivel_Proy_CHoCH_Alcista_{tf}"
                    key_ba = f"Nivel_Proy_CHoCH_Bajista_{tf}"
                nivel_alc = parse_float(data.get(key_al))
                nivel_baj = parse_float(data.get(key_ba))

                if nivel_alc is None and nivel_baj is None:
                    dist = None
                    direccion_nivel = None
                else:
                    dt = str(direccion_tendencia or "").lower()
                    if "alcista" in dt and nivel_alc is not None:
                        nivel, direccion_nivel = nivel_alc, "alcista"
                    elif "bajista" in dt and nivel_baj is not None:
                        nivel, direccion_nivel = nivel_baj, "bajista"
                    else:
                        nivel = nivel_alc if nivel_alc is not None else nivel_baj
                        direccion_nivel = "alcista" if nivel_alc is not None else "bajista"
                    dist = abs(valor_mcdo - nivel) if (valor_mcdo is not None and nivel is not None) else None

                valores_raw[ind] = {
                    "distancia": dist,
                    "atr":        atr,
                    "direccion_nivel":    direccion_nivel,
                    "direccion_tendencia": direccion_tendencia,
                    "caso": None,
                }






            elif ind == "BBWIDTH":
                valores_raw[ind] = {
                    "bb_width": data.get(f"BB_Width_{tf}", 0.0),
                    "tf": tf
                }

            elif ind == "VOLREL":
                for alt in (f"VolRel_{tf}", f"VolRel {tf}", f"VOLREL_{tf}"):
                    if alt in data:
                        valores_raw[ind] = data[alt]
                        break
                else:
                    valores_raw[ind] = 0.0

            else:
                # Todos los demás indicadores (RSI, ATR, DIST_ATH_LE_ATR, etc.)
                valores_raw[ind] = data.get(f"{ind}_{tf}", None)

        # ─────────── Calcula score ponderado ────────────
        fila_eval = {"indicadores": list(pesos), "valores": valores_raw}
        score_bruto, detalles = calcular_score_final(fila_eval, clave, tf, data)

        # ─────────── Construye fila de salida ────────────
        row = {
            "Activo":          activo,
            "TF":              tf,
            "Direccion":       data.get(f"frame_direccion_{tf}", ""),
            "Tipo de Entrada": data.get(f"tipo_entrada_{tf}", tipo_ent),
        }
        # añade Score×peso por indicador
        for ind, peso in pesos.items():
            val = detalles.get(ind, 0)
            if isinstance(val, (int, float)):
                row[ind] = f"<b>Score×peso: {round(val, 2)}</b>"
            else:
                row[ind] = f"<b>Score×peso: {val}</b>"  # puede ser texto como "Excluido"

        # Ajuste por ATR
        atr    = float(data.get(f"ATR_{tf}", 0.0))
        factor = 1.0 if atr > 0.8 else (0.9 if atr >= 0.4 else 0.8)
        row["ATR"] = f"<b>Factor: {factor}</b>"

        # Score final y umbral dinámico
        final_pct   = round(score_bruto * factor * 100, 2)
        um_dinamico = round(get_umbral(clave, factor), 2)
        row["Score Final TF"]  = f"{final_pct} %"
        row["Umbral dinámico"] = f"{um_dinamico} %"

        # Estado técnico
        if final_pct >= um_dinamico:
            data[f"is_valid_{tf}"] = True
            row["Status Técnico"] = (
                f"<span style='color:green;font-weight:bold'>✔</span> "
                f"Validado técnicamente, {final_pct}% ≥ {um_dinamico}%"
            )
        else:
            row["Status Técnico"] = (
                f"<span style='color:red;font-weight:bold'>✘</span> "
                f"Invalidado técnicamente, {final_pct}% < {um_dinamico}%"
            )

        # ─────────── Mostrar tabla ────────────
        df = pd.DataFrame([row])
        cols_base  = ["Activo","TF","Direccion","Tipo de Entrada"]
        extras     = ["ATR","Score Final TF","Umbral dinámico","Status Técnico"]
        cols_indic = [c for c in df.columns if c not in cols_base + extras]
        df         = df[cols_base + cols_indic + extras]

        col_theme = colores_por_tf.get(tf, {"bg":"#AAA","fg":"#000","border":"#555"})
        display(HTML(f"<h2>Resultados Análisis Técnico – TF {tf}</h2>"))
        mostrar_tabla(df,
                      header_bg=col_theme["bg"],
                      header_fg=col_theme["fg"],
                      cell_border_color=col_theme["border"])
        display(HTML("<hr>"))






    # 2.10 Preparación previa a Bloque 3 =========================
    # 2.10.1 Construcción automática de pivotes para Bloque 3
    def construir_pivotes_desde_datos(data, tf):
        pivs = []
        for i in range(1, 4):
            r_key = f"Resistance_pivot_{i}_{tf}"
            s_key = f"Support_pivot_{i}_{tf}"
            if r_key in data:
                pivs.append({"valor": data[r_key], "tests": 4 - i})
            if s_key in data:
                pivs.append({"valor": data[s_key], "tests": 4 - i})
        data[f"pivotes_{tf}"] = pivs

    # 2.10.2 Flag explícita: qué TF son estructurales
    for tf in detectar_temporalidades(data):
        data[f"is_frame_estructural_{tf}"] = evaluar_frame_estructural(data, tf)


    # ======================================================================
    # 🔵 2.11 FILTRO FINAL DE TFs VALIDADOS PARA BLOQUE 3
    #      ––– Solo los TF que
    #            ① fueron validados en Bloque 1  (ya están en valid_tfs)
    #         y ② superaron el umbral técnico en Bloque 2 (data['is_valid_<tf>'])
    #        pasarán al Bloque 3.
    # ======================================================================
    valid_tfs_b3 = {tf for tf in valid_tfs if data.get(f"is_valid_{tf}", False)}


    # 2.12 Línea final divisoria visual
    display(HTML("<hr style='border:2px solid #000000; margin:12px 0;'>"))







    # ==============================
    # 3. BLOQUE 3 – POSICIONES (Entradas, SL, TP, RBB, % Win/Fail)
    # ==============================
    import pandas as pd
    from IPython.display import HTML, display


    # 3.1 Auxiliar para formatear enteros con separador de miles
    def formato_entero(x):
        try:
            i = int(round(float(x)))
            s = f"{i:,}"
            return s.replace(",", "X").replace(".", ",").replace("X", ".")
        except:
            return x

    # ——— 3.1bis Funciones de RBB y % Win/Fail ———
    def calcular_rbb(ent_vals, sl_vals, tp_vals, tipo_operacion):
        lines = []
        for i, (e, sl, tp) in enumerate(zip(ent_vals, sl_vals, tp_vals), start=1):
            if tp is None or sl is None or e is None:
                lines.append(f"RBB{i}→-")
                continue
            riesgo = abs(e - sl) or 1e-6
            # Esta es la corrección:
            if tipo_operacion == "long":
                beneficio = tp - e
            else:  # short
                beneficio = e - tp
            if riesgo == 0:
                lines.append(f"RBB{i}→1/∞")
                continue
            ratio = beneficio / riesgo
            if ratio <= 0:
                lines.append(f"RBB{i}→1/–")
                continue
            denom = round(ratio)
            lines.append(f"RBB{i}→1/{denom}")
        return "\n".join(lines)





    def calcular_af(ent_vals, sl_vals, tp_vals, tipo_operacion, tipo_tp, data, tf):
        lines = []
        for i, (e, sl, tp) in enumerate(zip(ent_vals, sl_vals, tp_vals), start=1):
            # Si falta algún valor, mostramos "-" en vez de calcular
            if tp is None or sl is None or e is None:
                lines.append(f"%A/F{i}→-")
                continue
            # --- Lógica original ---
            distancia_tp = abs(tp - e)
            distancia_sl = abs(sl - e)
            adx = data.get(f"ADX_{tf}", 20)
            score = data.get(f"Frame_Score_{tf}", 50)
            rsi = data.get(f"RSI_{tf}", 50)

            # Ejemplo de cálculo
            base = 70 if tipo_operacion == "long" else 65
            bonus_adx = (adx - 20) * 0.7
            bonus_score = (score - 50) * 0.4
            penaliza_tp = -distancia_tp * 0.04
            penaliza_rsi = -(abs(rsi - 50) * 0.1)

            pct_acierto = base + bonus_adx + bonus_score + penaliza_tp + penaliza_rsi
            pct_acierto = max(35, min(95, int(pct_acierto)))  # Rango seguro

            penaliza_sl = distancia_sl * 0.03
            pct_fallo = 100 - pct_acierto - 7 - penaliza_sl
            pct_fallo = max(10, min(45, int(pct_fallo)))

            lines.append(f"%A/F{i}→{pct_acierto}%/{pct_fallo}%")
        return "\n".join(lines)





    # 3.2 Función de Entradas según “Flujo Entradas Múltiples”
    def calcular_entradas(data, tf):
        rt      = data.get("Valor_mcdo", 0)
        atr     = data.get(f"ATR_{tf}", 1)
        vol     = data.get(f"volatilidad_{tf}", "media")
        dir_    = data.get(f"direccion_{tf}", "alcista")
        estruc  = data.get(f"is_frame_estructural_{tf}", False)
        ath     = data.get(f"ath_break_{tf}", False)
        entrada = data.get(f"tipo_entrada_{tf}", "táctica")

        # 3.2.1 Separación dinámica (sep)
        if estruc:
            if dir_ == "alcista":
                sep = 0.30*atr if vol == "alta" else (0.25*atr if vol == "media" else 0.15*atr)
            elif dir_ == "bajista":
                sep = 0.35*atr if vol == "alta" else (0.30*atr if vol == "media" else 0.20*atr)
            else:
                # LATERAL ESTRUCTURAL: distinguir entre alcista y bajista
                lateral_dir = data.get(f"direccion_{tf}", "lateral")
                if lateral_dir == "lateral bajista":
                    sep = 0.40*atr if vol == "alta" else (0.35*atr if vol == "media" else 0.25*atr)
                else:
                    sep = 0.25*atr if vol == "alta" else (0.20*atr if vol == "media" else 0.10*atr)
        else:
            if dir_ == "alcista":
                sep = (0.35 if ath else 0.40)*atr if vol == "alta" else ((0.30 if ath else 0.35)*atr if vol == "media" else ((0.20 if ath else 0.25)*atr))
            elif dir_ == "bajista":
                sep = 0.35*atr if vol == "alta" else (0.30*atr if vol == "media" else 0.20*atr)
            else:
                sep = (0.35 if ath else 0.40)*atr if vol == "alta" else ((0.30 if ath else 0.35)*atr if vol == "media" else ((0.20 if ath else 0.25)*atr))


        # 3.2.2 Delta según tipo de entrada
        delta = 0
        if estruc:
            if dir_ == "alcista":
                delta = 0.20*atr if entrada=="agresiva" else 0.15*atr
            elif dir_ == "bajista":
                delta = 0.25*atr if entrada=="agresiva" else 0.20*atr
            else:  # lateral estructural
                if entrada == "táctica":
                    # distingue entre lateral alcista y bajista estructural
                    lateral_dir = data.get(f"direccion_{tf}", "lateral")
                    if lateral_dir == "lateral alcista":
                        delta = 0.10*atr
                    elif lateral_dir == "lateral bajista":
                        delta = 0.25*atr
                    else:
                        delta = 0.10*atr  # fallback
                else:
                    delta = 0.10*atr  # fallback si llega entrada inválida

        else:
            if dir_ == "alcista":
                if ath:
                    if entrada == "táctica":
                        delta = 0.25*atr
                    elif entrada == "agresiva":
                        delta = 0.30*atr
                    else:
                        delta = 0.35*atr  # conservadora

                else:
                    delta = 0.25*atr if entrada=="táctica" else 0.30*atr
            elif dir_ == "bajista":
                delta = 0.40*atr
            else:
                # LATERAL PROYECTADO: distinguir si es táctica sin ATH
                if ath and entrada == "táctica":
                    delta = 0.25*atr
                elif entrada == "táctica":
                    delta = 0.30*atr
                elif entrada == "agresiva":
                    delta = 0.35*atr
                else:
                    delta = 0.35*atr  # fallback conservador/agresivo anticipada


        # 3.2.3 Ajuste de signo si bajista
        sign = -1 if dir_=="bajista" else 1
        delta *= sign
        sep   *= sign

        # 3.2.4 Cálculo de niveles E1, E2, E3
        E1 = rt + delta
        E2 = E1 + sep
        E3 = E2 + sep

        # 3.2.4 Cálculo de niveles E1, E2, E3
        ...
        vals = [E1, E2, E3]
        txt  = "\n".join(f"E{i}→{formato_entero(vals[i-1])}" for i in (1,2,3))
        return txt, vals, dir_


    # 3.3 Función Stop Loss con pivotes y fallback
    def calcular_stop_loss(data, tf, ent_vals, tipo):
        atr     = data.get(f"ATR_{tf}", 1)
        vol     = data.get(f"volatilidad_{tf}", "media")
        estruc  = data.get(f"is_frame_estructural_{tf}", False)
        ath     = data.get(f"ath_break_{tf}", False)
        entrada = data.get(f"tipo_entrada_{tf}", "táctica")
        pivotes = data.get(f"pivotes_{tf}", [])

        # 3.3.1 Rango dinámico según flujo (con lateral alcista/bajista)
        lateral_dir = data.get(f"direccion_{tf}", "lateral")

        if tipo == "alcista":
            if estruc:
                min_r, max_r = (0.5*atr, 1.5*atr) if vol=="alta" else \
                              ((0.4*atr, 1.2*atr) if vol=="media" else (0.3*atr, 0.9*atr))
            else:
                # Proyectado alcista con/sin ATH
                if ath:
                    min_r, max_r = (0.7*atr, 2.0*atr) if vol == "alta" else \
                                  ((0.6*atr, 1.8*atr) if vol == "media" else (0.4*atr, 1.3*atr))
                else:
                    min_r, max_r = (0.6*atr, 1.8*atr) if vol == "alta" else \
                                  ((0.5*atr, 1.5*atr) if vol == "media" else (0.3*atr, 1.0*atr))

        elif tipo == "bajista":
            if estruc:
                min_r, max_r = (0.6*atr, 1.8*atr) if vol=="alta" else \
                              ((0.5*atr, 1.5*atr) if vol=="media" else (0.3*atr, 1.0*atr))
            else:
                # Proyectado bajista (nunca ATH)
                min_r, max_r = (0.7*atr, 2.0*atr) if vol=="alta" else \
                              ((0.6*atr, 1.8*atr) if vol=="media" else (0.4*atr, 1.3*atr))

        elif lateral_dir == "lateral alcista":
            if estruc:
                min_r, max_r = (0.5*atr, 1.5*atr) if vol=="alta" else \
                              ((0.4*atr, 1.2*atr) if vol=="media" else (0.3*atr, 0.9*atr))
            else:
                # Proyectado lateral alcista con/sin ATH
                if ath:
                    min_r, max_r = (0.6*atr, 1.8*atr) if vol=="alta" else \
                                  ((0.5*atr, 1.5*atr) if vol=="media" else (0.4*atr, 1.2*atr))
                else:
                    min_r, max_r = (0.7*atr, 2.0*atr) if vol=="alta" else \
                                  ((0.6*atr, 1.8*atr) if vol=="media" else (0.5*atr, 1.5*atr))

        else:  # lateral bajista
            if estruc:
                min_r, max_r = (0.6*atr, 1.8*atr) if vol=="alta" else \
                              ((0.5*atr, 1.5*atr) if vol=="media" else (0.3*atr, 1.0*atr))
            else:
                # Proyectado lateral bajista (nunca ATH)
                min_r, max_r = (0.7*atr, 2.0*atr) if vol=="alta" else \
                              ((0.6*atr, 1.8*atr) if vol=="media" else (0.4*atr, 1.3*atr))

        # 3.3.2 Factores de fallback
        # Inicializamos siempre las tres variables
        fact_ag = fact_ta = fact_co = None
        lateral_dir = data.get(f"direccion_{tf}", "lateral")

        if estruc:
            # Estructural
            if tipo == "alcista":
                fact_ag, fact_ta, fact_co = 0.20, 0.15, None
            elif tipo == "bajista":
                fact_ag, fact_ta, fact_co = 0.25, 0.20, None
            elif lateral_dir == "lateral alcista":
                fact_ag, fact_ta, fact_co = None, 0.10, None
            else:  # lateral bajista estructural
                fact_ag, fact_ta, fact_co = None, 0.25, None
        else:
            # Proyectado
            if tipo == "alcista":
                if ath:
                    fact_ag, fact_ta, fact_co = 0.25, 0.20, 0.30
                else:
                    fact_ag, fact_ta, fact_co = 0.30, 0.25, 0.35
            elif tipo == "bajista":
                fact_ag, fact_ta, fact_co = None, None, 0.35
            elif lateral_dir == "lateral alcista":
                if ath:
                    fact_ag, fact_ta, fact_co = 0.30, 0.25, 0.35
                else:
                    fact_ag, fact_ta, fact_co = 0.35, 0.30, None
            else:  # lateral bajista proyectado
                fact_ag, fact_ta, fact_co = None, None, 0.40

        # Selección del factor según tipo de entrada
        if entrada == "agresiva" and fact_ag is not None:
            factor = fact_ag
        elif entrada == "táctica" and fact_ta is not None:
            factor = fact_ta
        else:
            factor = fact_co


        # 3.3.3 Filtrado de pivotes válidos
        def filtro(p):
            val   = p["valor"]
            ok_tr = (tipo=="alcista" and val < ent_vals[0]) or (tipo=="bajista" and val > ent_vals[0]) or (tipo=="lateral")
            dist  = abs(ent_vals[0] - val)
            return ok_tr and (min_r <= dist <= max_r)

        pivs = sorted(filter(filtro, pivotes), key=lambda x: -x["tests"])

        # 3.3.4 Generación de SL1/2/3
        SL = []
        base = ent_vals[0]
        for i in range(3):
            if i < len(pivs):
                sl = pivs[i]["valor"]
            else:
                if factor is None:
                    # Si no hay factor definido, usamos ATR mínimo del rango dinámico
                    factor = min_r/atr
                sl = base - factor*atr if tipo!="bajista" else base + factor*atr

            SL.append(sl)
            base = sl

        sl_txt = "SL1→" + formato_entero(SL[0]) + "<br>" + \
                "SL2→" + formato_entero(SL[1]) + "<br>" + \
                "SL3→" + formato_entero(SL[2])
        return sl_txt, SL


    def seleccionar_herramientas(direccion, tipo_frame, rompio_ath):
        # Devuelve la lista de nombres de niveles relevantes para cada rama
        if direccion == 'alcista':
            if tipo_frame == 'estructural':
                return ['ATH', 'Marco High', 'Fib 78.6%', 'Fib 100%', 'BB Upper', 'BOS Alcista', 'CHoCH Alcista', 'Mid Range']
            else:
                if rompio_ath:
                    return ['ATH', 'Marco High', 'Fib 78.6%', 'Fib 100%', 'BB Upper', 'BOS Alcista', 'CHoCH Alcista']
                else:
                    return ['Marco High', 'Fib 78.6%', 'Fib 100%', 'BB Upper', 'BOS Alcista', 'CHoCH Alcista']
        elif direccion == 'bajista':
            if tipo_frame == 'estructural':
                return ['Marco Low', 'Fib 21.4%', 'Fib 0%', 'BB Lower', 'BOS Bajista', 'CHoCH Bajista', 'Mid Range']
            else:
                return ['Marco Low', 'Fib 21.4%', 'Fib 0%', 'BB Lower', 'BOS Bajista', 'CHoCH Bajista']
        elif direccion == 'lateral alcista':
            if tipo_frame == 'estructural':
                return ['Marco High', 'Mid Range', 'Fib 61.8%', 'Fib 78.6%', 'Fib 100%', 'BB Upper', 'BOS Alcista', 'CHoCH Alcista']
            else:
                if rompio_ath:
                    return ['ATH', 'Marco High', 'Mid Range', 'Fib 78.6%', 'Fib 100%', 'BB Upper', 'BOS Alcista', 'CHoCH Alcista']
                else:
                    return ['Marco High', 'Mid Range', 'Fib 61.8%', 'Fib 78.6%', 'Fib 100%', 'BB Upper', 'BOS Alcista', 'CHoCH Alcista']
        elif direccion == 'lateral bajista':
            if tipo_frame == 'estructural':
                return ['Marco Low', 'Mid Range', 'Fib 21.4%', 'Fib 0%', 'BB Lower', 'BOS Bajista', 'CHoCH Bajista']
            else:
                return ['Marco Low', 'Mid Range', 'Fib 21.4%', 'Fib 0%', 'BB Lower', 'BOS Bajista', 'CHoCH Bajista']
        else:
            return []

    def calcular_tp_zona_tecnica(
        direccion,
        tipo_frame,
        rompio_ath,
        tipo_entrada,
        precio_entrada,
        niveles_tecnicos, # dict {nombre_nivel: valor}
        cantidad_tp=3,
        tolerancia=5,
        riesgo=10,
        max_mult_atr=3,
        atr=10
    ):
        """
        Calcula los Take Profits técnicos más relevantes según reglas robustas.
        """
        # 1. Selecciona herramientas según rama (puedes usar tu función anterior)
        herramientas_rama = seleccionar_herramientas(
            direccion, tipo_frame, rompio_ath
        )

        # 2. Extrae los niveles y los filtra por dirección
        candidatos = []
        for nombre in herramientas_rama:
            valor = niveles_tecnicos.get(nombre)
            if valor is None:
                continue

            # ----------  CONVERSIÓN TEXTO → FLOAT ----------
            if isinstance(valor, str):
                # quita puntos de miles y cambia coma decimal
                tmp = valor.replace('.', '').replace(',', '.').strip()
                try:
                    valor = float(tmp)
                except ValueError:
                    # no es numérico (∅, -, etc.) → ignoramos este nivel
                    continue    # ← sigue dentro del for
            # -----------------------------------------------

            # ahora `valor` es numérico (float o int)  ➜ filtramos por dirección
            if direccion in ('alcista', 'lateral alcista') and valor > precio_entrada:
                candidatos.append({"nombre": nombre, "valor": valor})
            elif direccion in ('bajista', 'lateral bajista') and valor < precio_entrada:
                candidatos.append({"nombre": nombre, "valor": valor})

        # 3. Filtra por distancia mínima (ratio 1:1)
        candidatos = [n for n in candidatos if abs(n["valor"] - precio_entrada) >= riesgo]

        # 4. Agrupa confluencias (si hay 2+ niveles en ±tolerancia)
        usados = set()
        grupos = []
        for i, ni in enumerate(candidatos):
            if i in usados: continue
            grupo = [ni]
            for j, nj in enumerate(candidatos[i+1:], start=i+1):
                if abs(ni["valor"] - nj["valor"]) <= tolerancia:
                    grupo.append(nj)
                    usados.add(j)
            grupos.append(grupo)

        # 5. Toma promedio (o el valor de mayor jerarquía si prefieres) del grupo como TP
        tp_candidates = []
        for grupo in grupos:
            if len(grupo) == 1:
                tp_candidates.append(grupo[0]["valor"])
            else:
                avg_valor = sum(n["valor"] for n in grupo) / len(grupo)
                tp_candidates.append(avg_valor)

        # 6. Filtra por rango máximo (ej: 3*ATR)
        tp_candidates = [tp for tp in tp_candidates if abs(tp - precio_entrada) <= max_mult_atr * atr]

        # 7. Ordena por distancia a la entrada
        tp_candidates.sort(key=lambda x: abs(x - precio_entrada))

        # 8. Aplica distancia mínima entre TPs
        tp_final = []
        for tp in tp_candidates:
            if all(abs(tp - prev) >= riesgo for prev in tp_final):
                tp_final.append(tp)
            if len(tp_final) == cantidad_tp:
                break

        # 9. Completa con “–” si faltan TPs
        while len(tp_final) < cantidad_tp:
            tp_final.append(None)   # Usa None, o "-" según tu visualización

        # 10. REGLA FINAL: si solo hay un TP válido, solo ese y los demás "–"
        if tp_final and tp_final[0] is not None:
            if all(x is None for x in tp_final[1:]):
                return [tp_final[0], None, None]
            if tp_final[1] is None:
                return [tp_final[0], None, None]
            if tp_final[2] is None:
                return [tp_final[0], tp_final[1], None]
            return tp_final
        else:
            return [None, None, None]



    def calcular_take_profit(data, tf, ent_vals):
        atr       = data.get(f"ATR_{tf}", 1)
        estruc    = data.get(f"is_frame_estructural_{tf}", False)
        ath       = data.get(f"ath_break_{tf}", False)
        entrada   = data.get(f"tipo_entrada_{tf}", "táctica")
        direccion = data.get(f"direccion_{tf}", "lateral").lower()
        pivotes   = data.get(f"pivotes_{tf}", [])

        # —— NORMALIZACIÓN de la dirección ——
        if "lateral" in direccion and "alcista" in direccion:
            direccion = "lateral alcista"
        elif "lateral" in direccion and "bajista" in direccion:
            direccion = "lateral bajista"
        elif "tendencia" in direccion and "alcista" in direccion:
            direccion = "alcista"
        elif "tendencia" in direccion and "bajista" in direccion:
            direccion = "bajista"
        # si no encaja, se queda tal cual

        # 1. TP_PIVS: niveles por pivotes
        def filtrar_pivotes(pivs, base):
            if direccion == 'alcista' or direccion == 'lateral alcista':
                return [p for p in pivs if p["valor"] > base]
            elif direccion == 'bajista' or direccion == 'lateral bajista':
                return [p for p in pivs if p["valor"] < base]
            else:
                return [p for p in pivs]
        TP_PIVS = []
        base_p = ent_vals[0]
        for i in range(3):
            pivs = filtrar_pivotes(pivotes, base_p)
            if pivs:
                pivs_sorted = sorted(pivs, key=lambda x: -x.get("tests", 0))
                TPp = pivs_sorted[0]["valor"]
            else:
                TPp = None
            TP_PIVS.append(TPp)
            base_p = TPp if TPp is not None else base_p

        # 2. TP_ATR: niveles por ATR
        FACTORES = {
            ('alcista', True, 'agresiva'):   [1.30, 2.00, 2.80],
            ('alcista', True, 'táctica'):    [1.20, 1.80, 2.50],
            ('bajista', True, 'agresiva'):   [1.30, 2.00, 2.80],
            ('bajista', True, 'táctica'):    [1.20, 1.80, 2.40],
            ('lateral alcista', True, 'táctica'): [1.10, 1.60, None],
            ('lateral bajista', True, 'táctica'): [1.10, 1.60, None],
            # Frame proyectado: rompió ATH
            ('alcista', False, 'agresiva', True):  [1.40, 2.20, 3.00],
            ('alcista', False, 'táctica', True):   [1.35, 2.00, 2.80],
            ('alcista', False, 'conservadora', True): [1.30, 1.90, 2.60],
            ('lateral alcista', False, 'agresiva', True): [1.30, 2.00, None],
            ('lateral alcista', False, 'táctica', True):  [1.25, 1.80, None],
            ('lateral alcista', False, 'conservadora', True): [1.20, 1.70, None],
            # Frame proyectado: no rompió ATH
            ('alcista', False, 'agresiva', False): [1.00, 2.00, 2.80],
            ('alcista', False, 'táctica', False):  [1.25, 1.70, 2.40],
            ('lateral alcista', False, 'agresiva', False): [1.10, 1.70, None],
            ('lateral alcista', False, 'táctica', False):  [1.05, 1.50, None],
            # Frame proyectado bajista conservadora
            ('bajista', False, 'conservadora', True): [1.30, 2.10, 2.80],
            ('bajista', False, 'conservadora', False): [1.30, 2.10, 2.80],
            ('lateral bajista', False, 'conservadora', True): [1.20, 1.80, None],
            ('lateral bajista', False, 'conservadora', False): [1.20, 1.80, None]
        }
        if estruc:
            factors = FACTORES.get((direccion, True, entrada), [1.0, 1.5, 2.0])
        else:
            if direccion in ['alcista', 'lateral alcista']:
                factors = FACTORES.get((direccion, False, entrada, ath), [1.0, 1.5, 2.0])
            else:
                factors = FACTORES.get((direccion, False, 'conservadora', ath), [1.0, 1.5, 2.0])
        TP_ATR = []
        base_a = ent_vals[0]
        for i in range(3):
            factor = factors[i] if i < len(factors) and factors[i] is not None else 1.0
            if direccion in ['bajista', 'lateral bajista']:
                TPa = base_a - factor*atr
            else:
                TPa = base_a + factor*atr
            TP_ATR.append(TPa)
            base_a = TPa

        # 3. TP_ZONA: zona técnica por confluencias
        niveles_tecnicos = {
            'ATH': data.get(f"ATH_{tf}", None),
            'Marco High': data.get(f"Marco_High_{tf}", None),
            'Marco Low': data.get(f"Marco_Low_{tf}", None),
            'Fib 78.6%': data.get(f"Fib_78.6%_{tf}", None),
            'Fib 100%': data.get(f"Fib_100%_{tf}", None),
            'Fib 21.4%': data.get(f"Fib_21.4%_{tf}", None),
            'Fib 0%': data.get(f"Fib_0%_{tf}", None),
            'BB Upper': data.get(f"BB_Upper_{tf}", None),
            'BB Lower': data.get(f"BB_Lower_{tf}", None),
            'BOS Alcista': data.get(f"Nivel_Proy_BOS_Alcista_{tf}", None),
            'BOS Bajista': data.get(f"Nivel_Proy_BOS_Bajista_{tf}", None),
            'CHoCH Alcista': data.get(f"Nivel_Proy_CHoCH_Alcista_{tf}", None),
            'CHoCH Bajista': data.get(f"Nivel_Proy_CHoCH_Bajista_{tf}", None),
            'Mid Range': data.get(f"Mid_Range_{tf}", None),
        }
        if direccion in ["lateral alcista", "lateral bajista"]:
            cantidad_tp_zona = 2
        else:
            cantidad_tp_zona = 3
        TP_ZONA_confluencia = calcular_tp_zona_tecnica(
            direccion=direccion,
            tipo_frame='estructural' if estruc else 'proyectado',
            rompio_ath=ath,
            tipo_entrada=entrada,
            precio_entrada=ent_vals[0],
            niveles_tecnicos=niveles_tecnicos,
            cantidad_tp=cantidad_tp_zona,
            tolerancia=5
        )
        TP_ZONA = [tp if tp is not None else None for tp in TP_ZONA_confluencia] + [None]*max(0, 3-len(TP_ZONA_confluencia))


        # 4. tp_txt
        tp_txt = "\n".join([
            f"TP{i+1}→{formato_entero(TP_PIVS[i]) if TP_PIVS[i] is not None else '-'}"
            for i in range(3)
        ])

        return tp_txt, TP_PIVS, TP_ZONA, TP_ATR




    # 3.3ter Funciones de Score técnico para % Acierto (TP) y % Fallo (SL)
    def calcular_score_acierto(data, tf, tp_idx, tipo_operacion, tipo_tp):
        frame_score = data.get(f"Frame Score {tf}", 0)
        adx = data.get(f"ADX {tf}", 0)
        rsi = data.get(f"RSI {tf}", 50)
        macd_hist = data.get(f"MACD Hist {tf}", 0)
        obv = data.get(f"OBV Score {tf}", 0)
        volrel = data.get(f"VolRel {tf}", 0)
        ema_cross = data.get(f"EMA Cross {tf}", 0)
        obstacles = data.get(f"obstaculos_{tipo_tp}_tp{tp_idx+1}_{tf}", 0)
        zona_fav = data.get(f"zona_fav_{tf}", 0)

        score = 0
        if tipo_operacion == 'long':
            if frame_score > 0: score += 2
            if adx > 25: score += 1.5
            if rsi > 50: score += 1
            if macd_hist > 0: score += 1
            if obv > 0.7: score += 1
            if ema_cross == 1: score += 1
            if zona_fav == 1: score += 0.5
        else:
            if frame_score < 0: score += 2
            if adx > 25: score += 1.5
            if rsi < 50: score += 1
            if macd_hist < 0: score += 1
            if obv < 0.3: score += 1
            if ema_cross == -1: score += 1
            if zona_fav == 1: score += 0.5
        score -= obstacles * 1.2
        if volrel < 0.5: score += 0.5
        if tp_idx == 2: score -= 1

        pct = int(max(55, min(95, 55 + score*6)))
        return pct

    def calcular_score_fallo(data, tf, sl_idx, tipo_operacion):
        frame_score = data.get(f"Frame Score {tf}", 0)
        adx = data.get(f"ADX {tf}", 0)
        rsi = data.get(f"RSI {tf}", 50)
        macd_hist = data.get(f"MACD Hist {tf}", 0)
        obv = data.get(f"OBV Score {tf}", 0)
        volrel = data.get(f"VolRel {tf}", 0)
        ema_cross = data.get(f"EMA Cross {tf}", 0)
        near_obstacle = data.get(f"sl_near_obstacle_{sl_idx+1}_{tf}", 0)
        zona_riesgo = data.get(f"zona_riesgo_{tf}", 0)

        score = 0
        if tipo_operacion == 'long':
            if frame_score < 0: score += 2
            if adx < 20: score += 1
            if rsi < 40: score += 1
            if macd_hist < 0: score += 1
            if obv < 0.3: score += 1
            if ema_cross == -1: score += 1
            if zona_riesgo == 1: score += 0.5
        else:
            if frame_score > 0: score += 2
            if adx < 20: score += 1
            if rsi > 60: score += 1
            if macd_hist > 0: score += 1
            if obv > 0.7: score += 1
            if ema_cross == 1: score += 1
            if zona_riesgo == 1: score += 0.5
        score += near_obstacle * 1.2
        if volrel > 0.8: score += 0.5
        if sl_idx == 2: score -= 1

        pct = int(max(10, min(45, 10 + score*4)))
        return pct



    # ————————————————————————————————————————————————————————————————
    def generar_tabla_bloque3(data):
        """
        Crea la tabla final de posiciones usando SOLO los TF contenidos en
        valid_tfs_b3 (conjunto construido en la sección 2.11).
        """
        rows = []
        for tf in detectar_temporalidades(data):
            if tf not in valid_tfs_b3:          # ← descarta TF no validados
                continue

            # ——————————————————————————————————
            # 3.4.0 Normalización de la dirección
            raw_dir = data.get(f"direccion_{tf}", "").lower()
            if "lateralidad con sesgo alcista" in raw_dir or "lateral sesgo alcista" in raw_dir:
                dir_simple = "lateral alcista"
            elif "lateralidad con sesgo bajista" in raw_dir or "lateral sesgo bajista" in raw_dir:
                dir_simple = "lateral bajista"
            elif "tendencia alcista" in raw_dir:
                dir_simple = "alcista"
            elif "tendencia bajista" in raw_dir:
                dir_simple = "bajista"
            else:
                dir_simple = raw_dir.replace("tendencia ", "").strip()

            # 3.4.1 Variables básicas
            tipo_operacion = "long" if dir_simple in ["alcista", "lateral alcista"] else "short"
            tipo_frame     = "Frame estructural" if data[f"is_frame_estructural_{tf}"] else "Frame proyectado"

            # 3.4.2 Recuperar tipo de entrada
            tipo_ent = data.get(f"tipo_entrada_{tf}", "-")

            # 3.4.3 Frame low/high (estructural vs proyectado)
            if data[f"is_frame_estructural_{tf}"]:
                low_str  = formato_entero(data.get(f"Marco_Low_{tf}", 0))
                high_str = formato_entero(data.get(f"Marco_High_{tf}", 0))
            else:
                info     = determinar_frame_proyectado(data, tf)
                low_str  = info["f_low"]
                high_str = info["f_high"]

            base = {
                "Activo":       data.get("Activo", "-"),
                "TF":           tf,
                "Direccion":    dir_simple,
                "Tipo Frame":   tipo_frame,
                "Tipo Entrada": tipo_ent,
                "Valor actual": formato_entero(data.get("Valor_mcdo", 0)),
                "Frame low":    low_str,
                "Frame high":   high_str
            }

            # 3.4.4 Cálculo de Entradas, SL y TP
            ent_txt, ent_vals, tp_type = calcular_entradas(data, tf)
            sl_txt,  sl_vals            = calcular_stop_loss(data, tf, ent_vals, tp_type)
            tp_txt,  tp_p, tp_z, tp_a   = calcular_take_profit(
                                            { **data, f"direccion_{tf}": dir_simple },
                                            tf, ent_vals
                                        )

            # 3.4.5 Ajuste de índices y textos
            piv_vals = [tp_p[i] if tp_p[i] is not None else None for i in range(3)]
            zon_vals = [tp_z[i] if tp_z[i] is not None else None for i in range(3)]
            atr_vals = [tp_a[i] if tp_a[i] is not None else None for i in range(3)]

            def rbb_af_or_dash(ent, sl, tp, tipo_operacion, tipo_tp, data, tf):
                if all(x is None for x in tp):
                    return ["-"]*3, ["-"]*3
                return (
                    calcular_rbb(ent, sl, tp, tipo_operacion).split("\n"),
                    calcular_af(ent, sl, tp, tipo_operacion, tipo_tp, data, tf).split("\n")
                )

            rbb_piv, af_piv = rbb_af_or_dash(ent_vals, sl_vals, piv_vals, tipo_operacion, "pivote", data, tf)
            rbb_zon, af_zon = rbb_af_or_dash(ent_vals, sl_vals, zon_vals, tipo_operacion, "tecnico", data, tf)
            rbb_atr, af_atr = rbb_af_or_dash(ent_vals, sl_vals, atr_vals, tipo_operacion, "atr",     data, tf)

            row = {
                **base,
                "Entrada":           ent_txt.replace("\n","<br>"),
                "SL":                sl_txt.replace("\n","<br>"),
                "TP Pivots":         "<br>".join(f"TP{i+1}→{formato_entero(piv_vals[i]) if piv_vals[i] is not None else '-'}"
                                                for i in range(3)),
                "RBB Pivots":        "<br>".join(rbb_piv),
                "% A/F Pivots":      "<br>".join(af_piv),
                "TP zona Técnica":   "<br>".join(f"TP{i+1}→{formato_entero(zon_vals[i]) if zon_vals[i] is not None else '-'}"
                                                for i in range(3)),
                "RBB Zona":          "<br>".join(rbb_zon),
                "% A/F Zona":        "<br>".join(af_zon),
                "TP ATR factor":     "<br>".join(f"TP{i+1}→{formato_entero(atr_vals[i]) if atr_vals[i] is not None else '-'}"
                                                for i in range(3)),
                "RBB ATR":           "<br>".join(rbb_atr),
                "% A/F ATR":         "<br>".join(af_atr),
            }

            rows.append(row)

        return rows
    # ————————————————————————————————————————————————————————————————





    # 3.5 Ejecución y visualización de tabla final Bloque 3
    tabla3 = generar_tabla_bloque3(data)
    df3    = pd.DataFrame(
        tabla3,
        columns=[
            "Activo","TF","Direccion","Tipo Frame","Tipo Entrada","Valor actual",
            "Frame low","Frame high",
            "Entrada","SL",
            "TP Pivots","RBB Pivots","% A/F Pivots",
            "TP zona Técnica","RBB Zona","% A/F Zona",
            "TP ATR factor","RBB ATR","% A/F ATR"
        ]
    )

    # --- bloqueo de saltos en Entrada, SL y TP Pivots ---
    for col in ["Entrada", "SL", "TP Pivots", "TP zona Técnica", "TP ATR factor", "RBB Pivots", "% A/F Pivots", "RBB Zona", "% A/F Zona", "RBB ATR", "% A/F ATR"]:
        # cambia " flecha " por " NBSP-flecha-NBSP "
        df3[col] = df3[col].str.replace("→", "\u00A0→\u00A0", regex=False)
    # ----------------------------------------------------


    display(HTML("<h2 style='color:#8B4513;'>Bloque 3 – Posiciones (Entradas, SL, TP, RBB, % Win/Fail)</h2>"))
    if not df3.empty:
        mostrar_tabla(df3,
                      header_bg="#8B4513",
                      header_fg="#fff",
                      cell_border_color="#444")
    else:
        display(HTML("<p>No hay TFs válidos para Bloque 3</p>"))






In [33]:
# ==========================================
# │ INTERFAZ: subir Excel + elegir pestaña │
#   (todo aparece junto a las tablas)
# ==========================================
import io, pandas as pd, ipywidgets as w
from IPython.display import display, clear_output

up_loader  = w.FileUpload(accept='.xlsx,.xls', multiple=False)
dd_sheets  = w.Dropdown(options=[], description='Hoja:')
btn_run    = w.Button(description='Ejecutar análisis', button_style='success')
out_box    = w.Output()

# ── 1. Cargar nombres de hojas cuando subes el archivo ───────────────────
def _on_upload(change):
    dd_sheets.options = []
    if not up_loader.value:   # nada cargado
        return
    up  = next(iter(up_loader.value.values()))
    xl  = pd.ExcelFile(io.BytesIO(up['content']))
    dd_sheets.options = xl.sheet_names
    dd_sheets.value   = xl.sheet_names[0]     # pre-selecciona la primera hoja

up_loader.observe(_on_upload, names='value')

# ── 2. Ejecutar tu mega-script y mostrar resultados justo debajo ──────────
def _run_analysis(b):
    with out_box:
        clear_output()
        if not up_loader.value:
            print("⚠️  Sube un archivo primero"); return
        if not dd_sheets.value:
            print("⚠️  Selecciona una hoja"); return

        up   = next(iter(up_loader.value.values()))
        df   = pd.read_excel(io.BytesIO(up['content']),
                             sheet_name=dd_sheets.value,
                             header=None)

        # —— Construimos el texto clave/valor ————————————————
        if df.shape[1] >= 2:               # 2 columnas: clave | valor
            filas = [f"{r[0]}\n{r[1]}" for r in df.values if len(r) >= 2]
        else:                              # 1 columna: clave, valor, clave, valor…
            col = df.iloc[:, 0].astype(str).tolist()
            filas = []
            for i in range(0, len(col), 2):
                k = col[i].strip()
                v = col[i+1].strip() if i+1 < len(col) else ""
                filas.append(f"{k}\n{v}")
        # —— Verificamos si la clave "Activo" está presente, si no la añadimos ——
        if not any(f.split("\n")[0].strip().lower() == "activo" for f in filas):
            filas.insert(0, f"Activo\n{dd_sheets.value}")

        texto = "\n".join(filas)
        analizar_hoja(texto)        # las tablas aparecen AQUÍ 🔽

btn_run.on_click(_run_analysis)

# ── 3. Dibujar la interfaz (el output se adhiere debajo) ──────────────────
display(w.VBox([
    w.Label("📂 Selecciona archivo Excel:"),
    up_loader,
    dd_sheets,
    btn_run,
    out_box          # ← las tablas quedan pegadas justo aquí
]))
